In [ ]:
### Run yolo pre-trained model on sample images.
# python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.25 --iou-thres 0.5 --source ~/cp_wks/datasets/IDD_FGD_L1/val/images/

In [1]:
import pickle
import numpy as np
import torch
from utils.metrics import ConfusionMatrix, ap_per_class, box_iou

def process_batch(detections, labels, iouv):
    """
    Return correct predictions matrix. Both sets of boxes are in (x1, y1, x2, y2) format.
    Arguments:
        detections (Array[N, 6]), x1, y1, x2, y2, conf, class
        labels (Array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (Array[N, 10]), for 10 IoU levels
    """
    correct = np.zeros((detections.shape[0], iouv.shape[0])).astype(bool)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    correct_class = labels[:, 0:1] == detections[:, 5]
    matches_iou0 = np.array([])
    for i in range(len(iouv)):
        x = torch.where((iou >= iouv[i]) & correct_class)  # IoU > threshold and classes match
        if x[0].shape[0]:
            matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
            if x[0].shape[0] > 1:
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                # matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
            correct[matches[:, 1].astype(int), i] = True
            if(i==0):
                matches_iou0 = matches.copy()
    return torch.tensor(correct, dtype=torch.bool, device=iouv.device), matches_iou0


# with open("../fastercnn-pytorch-training-pipeline/frcnn_L3test_outputs.pkl", 'rb') as file:
#     # A new file will be created
#     all_outputs = pickle.load(file)
# # len(all_outputs), all_outputs[0]

# pred_boxes = all_outputs[0][0]['boxes'].numpy()
# pred_confs = all_outputs[0][0]['scores'].numpy()
# pred_labels = all_outputs[0][0]['labels'].numpy()
# # pred_boxes.shape,pred_confs.shape,pred_labels.shape
# pred = np.concatenate((pred_boxes,np.vstack((pred_confs,pred_labels)).T),axis=1)
# # pred.shape
# pred = torch.tensor(pred)

# with open("../fastercnn-pytorch-training-pipeline/frcnn_L3test_targets.pkl", 'rb') as file:
#     # A new file will be created
#     all_targets = pickle.load(file)
# # len(all_targets), all_targets[0]

# gt_boxes = all_targets[0][0]['boxes'].numpy()
# gt_labels = all_targets[0][0]['labels'].numpy()
# # gt_boxes.shape,gt_labels.shape
# labels = np.concatenate((np.expand_dims(gt_labels,axis=-1),gt_boxes),axis=1)
# # labels.shape
# labels = torch.tensor(labels)

# iouv = np.linspace(0.5,0.95,num=10)
# # iouv
# iouv = torch.tensor(iouv)

# correct, matches_iou0 = process_batch(pred, labels, iouv)

In [2]:
with open("../fastercnn-pytorch-training-pipeline/frcnn_L3test_targets.pkl", 'rb') as file:
    # A new file will be created
    all_targets = pickle.load(file)
with open("../fastercnn-pytorch-training-pipeline/frcnn_L3test_outputs.pkl", 'rb') as file:
    # A new file will be created
    all_outputs = pickle.load(file)
    
iouv = np.linspace(0.5,0.95,num=10)
# iouv
iouv = torch.tensor(iouv)

stats=[]
for bi,ai in enumerate(all_outputs):
    for bii,abi in enumerate(list(ai.keys())):
        pred_boxes = all_outputs[bi][abi]['boxes'].numpy()
        pred_confs = all_outputs[bi][abi]['scores'].numpy()
        pred_labels = all_outputs[bi][abi]['labels'].numpy()
        # pred_boxes.shape,pred_confs.shape,pred_labels.shape
        pred = np.concatenate((pred_boxes,np.vstack((pred_confs,pred_labels)).T),axis=1)
        # pred.shape
        pred = torch.tensor(pred)
        
        gt_boxes = all_targets[bi][bii]['boxes'].numpy()
        gt_labels = all_targets[bi][bii]['labels'].numpy()
        # gt_boxes.shape,gt_labels.shape
        labels = np.concatenate((np.expand_dims(gt_labels,axis=-1),gt_boxes),axis=1)
        # labels.shape
        labels = torch.tensor(labels)

        correct, matches_iou0 = process_batch(pred, labels, iouv)

        stats.append((correct, pred[:, 4], pred[:, 5], labels[:, 0]))  # (correct, conf, pcls, tcls)
        
stats = [torch.cat(x, 0).cpu().numpy() for x in zip(*stats)]  # to numpy
print(stats[0].shape, stats[1].shape, stats[2].shape, stats[3].shape )

(22163, 10) (22163,) (22163,) (4891,)


In [3]:
####### First evaluate simple L3 mAP and check if it is mathcing with what frcnn code displays as output mAP
from pathlib import Path
from utils.metrics import ConfusionMatrix, ap_per_class, box_iou
## eval using stats variable first then proceed to next cell
###### Run this for final output
save_dir=Path('')
plots=True
l3_class_indexes = {'background_background_background':0,'autorickshaw_Atul_Atul': 1, 'autorickshaw_autorickshaw_autorickshaw': 2, 'autorickshaw_Bajaj_Bajaj': 3, 'autorickshaw_Mahindra_Mahindra': 4, 'autorickshaw_Piaggio_Piaggio': 5, 'autorickshaw_TVS_TVS': 6, 'bus_bus_bus': 7, 'car_Audi_A3': 8, 'car_Audi_Q3': 9, 'car_Bmw_2-Series-220d': 10, 'car_Bmw_3-Series': 11, 'car_Bmw_X1': 12, 'car_car_car': 13, 'car_Chevrolet_Aveo': 14, 'car_Chevrolet_Beat': 15, 'car_Chevrolet_Cruze': 16, 'car_Chevrolet_Enjoy': 17, 'car_Chevrolet_Spark': 18, 'car_Chevrolet_Tavera': 19, 'car_Fiat_Linea': 20, 'car_Fiat_PuntoEvo': 21, 'car_Force_TraxToofan': 22, 'car_Ford_Aspire': 23, 'car_Ford_Ecosport': 24, 'car_Ford_EcoSportTitanium': 25, 'car_Ford_Everest': 26, 'car_Ford_Fiesta': 27, 'car_Ford_Figo': 28, 'car_Ford_Ikon': 29, 'car_Hindustan_Ambassador': 30, 'car_Honda_Accord': 31, 'car_Honda_Amaze': 32, 'car_Honda_Brio': 33, 'car_Honda_Brv': 34, 'car_Honda_City': 35, 'car_Honda_Civic': 36, 'car_Honda_Cr-V': 37, 'car_Honda_Jazz': 38, 'car_Honda_Wr-V': 39, 'car_Hyundai_Accent': 40, 'car_Hyundai_Aura': 41, 'car_Hyundai_Creta': 42, 'car_Hyundai_Eon': 43, 'car_Hyundai_I10': 44, 'car_Hyundai_I20': 45, 'car_Hyundai_Santro': 46, 'car_Hyundai_Verna': 47, 'car_Hyundai_Xcent': 48, 'car_Jeep_Compass': 49, 'car_Jeep_Wrangler': 50, 'car_Mahindra_Bolero': 51, 'car_Mahindra_Reva': 52, 'car_Mahindra_Scorpio': 53, 'car_Mahindra_TUV300': 54, 'car_Mahindra_Verito': 55, 'car_Mahindra_XUV500': 56, 'car_Mahindra_Xylo': 57, 'car_MarutiSuzuki_1000': 58, 'car_MarutiSuzuki_Alto800': 59, 'car_MarutiSuzuki_AltoK10': 60, 'car_MarutiSuzuki_Baleno': 61, 'car_MarutiSuzuki_Celerio': 62, 'car_MarutiSuzuki_Ciaz': 63, 'car_MarutiSuzuki_Dzire': 64, 'car_MarutiSuzuki_Eeco': 65, 'car_MarutiSuzuki_Ertiga': 66, 'car_MarutiSuzuki_Esteem2000': 67, 'car_MarutiSuzuki_Ignis': 68, 'car_MarutiSuzuki_Omni': 69, 'car_MarutiSuzuki_Ritz': 70, 'car_MarutiSuzuki_S-Cross': 71, 'car_MarutiSuzuki_Swift': 72, 'car_MarutiSuzuki_SX4': 73, 'car_MarutiSuzuki_VitaraBrezza': 74, 'car_MarutiSuzuki_WagonR': 75, 'car_MarutiSuzuki_Zen': 76, 'car_Mercedes-Benz_A-Class': 77, 'car_Mercedes-Benz_AmgGt4-DoorCoupe': 78, 'car_Mercedes-Benz_C-Class': 79, 'car_Mercedes-Benz_E-Class': 80, 'car_Mercedes-Benz_G-Class': 81, 'car_Mercedes-Benz_Gla-Class': 82, 'car_Mercedes-Benz_Gls': 83, 'car_Mercedes-Benz_S-Class': 84, 'car_Mitsubishi_Lancer': 85, 'car_Nissan_Micra': 86, 'car_Nissan_Sunny': 87, 'car_Nissan_Terrano': 88, 'car_Renault_Duster': 89, 'car_Renault_Kwid': 90, 'car_Renault_Lodgy': 91, 'car_Renault_Logan': 92, 'car_Renault_Scala': 93, 'car_Skoda_Fabia': 94, 'car_Skoda_Octavia': 95, 'car_Skoda_Rapid': 96, 'car_Skoda_Superb': 97, 'car_TataMotors_Hexa': 98, 'car_TataMotors_Indica': 99, 'car_TataMotors_Indigo': 100,'car_TataMotors_Nano': 101, 'car_TataMotors_Nexon': 102, 'car_TataMotors_Safari': 103, 'car_TataMotors_Sumo': 104, 'car_TataMotors_Tiago': 105, 'car_TataMotors_Tigor': 106, 'car_TataMotors_Zest': 107, 'car_Toyota_Corolla': 108, 'car_Toyota_Etios': 109, 'car_Toyota_EtiosLiva': 110, 'car_Toyota_Fortuner': 111, 'car_Toyota_Innova': 112, 'car_Toyota_Qualis': 113, 'car_Volkswagen_Ameo': 114, 'car_Volkswagen_Jetta': 115, 'car_Volkswagen_Polo': 116, 'car_Volkswagen_Vento': 117, 'car_Volvo_Xc40': 118, 'car_Volvo_Xc60': 119, 'mini-bus_mini-bus_mini-bus': 120, 'motorcycle_Bajaj_Avenger': 121, 'motorcycle_Bajaj_CT100': 122, 'motorcycle_Bajaj_Discover': 123, 'motorcycle_Bajaj_Discover100': 124, 'motorcycle_Bajaj_Discover110': 125, 'motorcycle_Bajaj_Discover125': 126, 'motorcycle_Bajaj_Discover135': 127, 'motorcycle_Bajaj_Platina': 128, 'motorcycle_Bajaj_Pulsar150': 129, 'motorcycle_Bajaj_Pulsar180': 130, 'motorcycle_Bajaj_Pulsar200': 131, 'motorcycle_Bajaj_Pulsar220F': 132, 'motorcycle_Bajaj_PulsarNS200': 133, 'motorcycle_Bajaj_PulsarRS200': 134, 'motorcycle_Bajaj_V12': 135, 'motorcycle_Bajaj_V15': 136, 'motorcycle_Hero_Glamour': 137, 'motorcycle_Hero_HFDeluxe': 138, 'motorcycle_Hero_Hunk': 139, 'motorcycle_Hero_Passion': 140, 'motorcycle_Hero_PassionPlus': 141, 'motorcycle_Hero_PassionPro': 142, 'motorcycle_Hero_Splendor': 143, 'motorcycle_Hero_SuperSplendor': 144, 'motorcycle_Hero_XPulse200': 145, 'motorcycle_HeroHonda_CBZ': 146, 'motorcycle_HeroHonda_SplendorNXG': 147, 'motorcycle_Honda_CBHornet160R': 148, 'motorcycle_Honda_CBTwister': 149, 'motorcycle_Honda_Karizma': 150, 'motorcycle_Honda_KarizmaZMR': 151, 'motorcycle_Honda_Shine': 152, 'motorcycle_Honda_SP125': 153, 'motorcycle_Honda_StunnerCBF': 154, 'motorcycle_Honda_Unicorn': 155, 'motorcycle_KTM_Duke': 156, 'motorcycle_Mahindra_Centuro': 157, 'motorcycle_motorcycle_motorcycle': 158, 'motorcycle_RoyalEnfield_Bullet350': 159, 'motorcycle_RoyalEnfield_Bullet500': 160, 'motorcycle_RoyalEnfield_Classic350': 161, 'motorcycle_RoyalEnfield_Classic500': 162, 'motorcycle_RoyalEnfield_ContinentalGT650': 163, 'motorcycle_RoyalEnfield_Interceptor650': 164, 'motorcycle_RoyalEnfield_Meteor350': 165, 'motorcycle_RoyalEnfield_Thunderbird350': 166, 'motorcycle_RoyalEnfield_Thunderbird350X': 167, 'motorcycle_Suzuki_Gixxer': 168, 'motorcycle_Suzuki_Samurai': 169, 'motorcycle_Suzuki_Slingshot': 170, 'motorcycle_TVS_ApacheRR310': 171, 'motorcycle_TVS_ApacheRTR160': 172, 'motorcycle_TVS_ApacheRTR200': 173, 'motorcycle_TVS_Excel100': 174, 'motorcycle_TVS_ExcelHeavyDuty': 175, 'motorcycle_TVS_Sport': 176, 'motorcycle_TVS_StarCityPlus': 177, 'motorcycle_TVS_Victor': 178, 'motorcycle_TVS_XL100': 179, 'motorcycle_Yamaha_Crux': 180, 'motorcycle_Yamaha_Fazer': 181, 'motorcycle_Yamaha_FZ25': 182, 'motorcycle_Yamaha_FZS-FI': 183, 'motorcycle_Yamaha_FZ-V3': 184, 'motorcycle_Yamaha_Libero': 185, 'motorcycle_Yamaha_R15': 186, 'motorcycle_Yamaha_RX100': 187, 'scooter_Bajaj_Chetak': 188, 'scooter_Hero_Duet': 189, 'scooter_Hero_Maestro': 190, 'scooter_Hero_Pleasure': 191, 'scooter_Honda_Activa': 192, 'scooter_Honda_Aviator': 193, 'scooter_Honda_Dio': 194, 'scooter_Honda_Grazia': 195, 'scooter_Mahindra_Gusto': 196, 'scooter_scooter_scooter': 197, 'scooter_Suzuki_Access': 198, 'scooter_Suzuki_Burgman': 199, 'scooter_Suzuki_Swish': 200, 'scooter_TVS_Jupiter': 201, 'scooter_TVS_Ntorq': 202, 'scooter_TVS_Pep': 203, 'scooter_TVS_Streak': 204,'scooter_TVS_Wego': 205, 'scooter_TVS_Zest': 206, 'scooter_Vespa_VXL125': 207, 'scooter_Vespa_ZX125': 208, 'scooter_Yamaha_Fascino125': 209, 'scooter_Yamaha_RayZR': 210, 'truck_AshokLeyland_AshokLeyland': 211, 'truck_BharatBenz_BharatBenz': 212, 'truck_Eicher_Eicher': 213, 'truck_Mahindra_Mahindra': 214, 'truck_SML_SML': 215, 'truck_Tata_Tata': 216, 'truck_truck_truck': 217}
l3_class_indexes_names = list(l3_class_indexes.keys())
# l3_class_indexes_names = {k: v for k, v in enumerate(list(l3_class_indexes.keys()))}

# l3_class_indexes_names = ['background_background_background', 'autorickshaw_Atul_Atul', 'autorickshaw_Bajaj_Bajaj', 'autorickshaw_autorickshaw_autorickshaw', 'autorickshaw_Mahindra', 'autorickshaw_Others', 'autorickshaw_Piaggio', 'autorickshaw_TVS', 'bus', 'car_Audi_A3', 'car_Audi_Q3', 'car_Bmw_2-Series-220d', 'car_Bmw_3-Series', 'car_Bmw_X1', 'car_Chevrolet_Aveo', 'car_Chevrolet_Beat', 'car_Chevrolet_Cruze', 'car_Chevrolet_Enjoy', 'car_Chevrolet_Spark', 'car_Chevrolet_Tavera', 'car_Covered', 'car_Fiat_Linea', 'car_Fiat_PuntoEvo', 'car_Force_TraxToofan', 'car_Ford_Aspire', 'car_Ford_EcoSportTitanium', 'car_Ford_Ecosport', 'car_Ford_Everest', 'car_Ford_Fiesta', 'car_Ford_Figo', 'car_Ford_Ikon', 'car_Hindustan_Ambassador', 'car_Honda_Accord', 'car_Honda_Amaze', 'car_Honda_Brio', 'car_Honda_Brv', 'car_Honda_City', 'car_Honda_Civic', 'car_Honda_Cr-V', 'car_Honda_Jazz', 'car_Honda_Wr-V', 'car_Hyundai_Accent', 'car_Hyundai_Creta', 'car_Hyundai_Eon', 'car_Hyundai_I10', 'car_Hyundai_I20', 'car_Hyundai_Santro', 'car_Hyundai_Verna', 'car_Hyundai_Xcent', 'car_Jeep_Compass', 'car_Jeep_Wrangler', 'car_Mahindra_Bolero', 'car_Mahindra_Reva', 'car_Mahindra_Scorpio', 'car_Mahindra_TUV300', 'car_Mahindra_Verito', 'car_Mahindra_XUV500', 'car_Mahindra_Xylo', 'car_MarutiSuzuki_1000', 'car_MarutiSuzuki_Alto800', 'car_MarutiSuzuki_AltoK10', 'car_MarutiSuzuki_Baleno', 'car_MarutiSuzuki_Celerio', 'car_MarutiSuzuki_Ciaz', 'car_MarutiSuzuki_Dzire', 'car_MarutiSuzuki_Eeco', 'car_MarutiSuzuki_Ertiga', 'car_MarutiSuzuki_Esteem2000', 'car_MarutiSuzuki_Ignis', 'car_MarutiSuzuki_Omni', 'car_MarutiSuzuki_Ritz', 'car_MarutiSuzuki_S-Cross', 'car_MarutiSuzuki_SX4', 'car_MarutiSuzuki_Swift', 'car_MarutiSuzuki_VitaraBrezza', 'car_MarutiSuzuki_WagonR', 'car_MarutiSuzuki_Zen', 'car_Mercedes-Benz_A-Class', 'car_Mercedes-Benz_AmgGt4-DoorCoupe', 'car_Mercedes-Benz_C-Class', 'car_Mercedes-Benz_E-Class', 'car_Mercedes-Benz_G-Class', 'car_Mercedes-Benz_Gla-Class', 'car_Mercedes-Benz_S-Class', 'car_Nissan_Micra', 'car_Nissan_Sunny', 'car_Nissan_Terrano', 'car_Others', 'car_Renault_Duster', 'car_Renault_Kwid', 'car_Renault_Lodgy', 'car_Renault_Logan', 'car_Renault_Scala', 'car_Skoda_Fabia', 'car_Skoda_Octavia', 'car_Skoda_Rapid', 'car_Skoda_Superb', 'car_TataMotors_Hexa', 'car_TataMotors_Indica', 'car_TataMotors_Indigo', 'car_TataMotors_Nano', 'car_TataMotors_Nexon', 'car_TataMotors_Safari', 'car_TataMotors_Sumo', 'car_TataMotors_Tiago', 'car_TataMotors_Tigor', 'car_TataMotors_Zest', 'car_Toyota_Corolla', 'car_Toyota_Etios', 'car_Toyota_EtiosLiva', 'car_Toyota_Fortuner', 'car_Toyota_Innova', 'car_Toyota_Qualis', 'car_Volkswagen_Ameo', 'car_Volkswagen_Jetta', 'car_Volkswagen_Polo', 'car_Volkswagen_Vento', 'car_Volvo_Xc40', 'car_Volvo_Xc60', 'mini-bus_Others', 'motorcycle_Bajaj_Avenger', 'motorcycle_Bajaj_CT100', 'motorcycle_Bajaj_Discover', 'motorcycle_Bajaj_Discover100', 'motorcycle_Bajaj_Discover110', 'motorcycle_Bajaj_Discover125', 'motorcycle_Bajaj_Discover135', 'motorcycle_Bajaj_Platina', 'motorcycle_Bajaj_Pulsar150', 'motorcycle_Bajaj_Pulsar180', 'motorcycle_Bajaj_Pulsar200', 'motorcycle_Bajaj_Pulsar220F', 'motorcycle_Bajaj_PulsarNS200', 'motorcycle_Bajaj_PulsarRS200', 'motorcycle_Bajaj_V12', 'motorcycle_Bajaj_V15', 'motorcycle_Covered', 'motorcycle_HeroHonda_CBZ', 'motorcycle_HeroHonda_SplendorNXG', 'motorcycle_Hero_Glamour', 'motorcycle_Hero_HFDeluxe', 'motorcycle_Hero_Hunk', 'motorcycle_Hero_Passion', 'motorcycle_Hero_PassionPlus', 'motorcycle_Hero_PassionPro', 'motorcycle_Hero_Splendor', 'motorcycle_Hero_SuperSplendor', 'motorcycle_Hero_XPulse200', 'motorcycle_Honda_CBHornet160R', 'motorcycle_Honda_CBTwister', 'motorcycle_Honda_Karizma', 'motorcycle_Honda_KarizmaZMR', 'motorcycle_Honda_SP125', 'motorcycle_Honda_Shine', 'motorcycle_Honda_StunnerCBF', 'motorcycle_Honda_Unicorn', 'motorcycle_KTM_Duke', 'motorcycle_Mahindra_Centuro', 'motorcycle_Others', 'motorcycle_RoyalEnfield_Bullet350', 'motorcycle_RoyalEnfield_Bullet500', 'motorcycle_RoyalEnfield_Classic350', 'motorcycle_RoyalEnfield_Classic500', 'motorcycle_RoyalEnfield_ContinentalGT650', 'motorcycle_RoyalEnfield_Interceptor650', 'motorcycle_RoyalEnfield_Meteor350', 'motorcycle_RoyalEnfield_Thunderbird350', 'motorcycle_RoyalEnfield_Thunderbird350X', 'motorcycle_Suzuki_Gixxer', 'motorcycle_Suzuki_Samurai', 'motorcycle_Suzuki_Slingshot', 'motorcycle_TVS_ApacheRTR160', 'motorcycle_TVS_ApacheRTR200', 'motorcycle_TVS_Excel100', 'motorcycle_TVS_ExcelHeavyDuty', 'motorcycle_TVS_Sport', 'motorcycle_TVS_StarCityPlus', 'motorcycle_TVS_Victor', 'motorcycle_TVS_XL100', 'motorcycle_Yamaha_Crux', 'motorcycle_Yamaha_FZ-V3', 'motorcycle_Yamaha_FZ25', 'motorcycle_Yamaha_FZS-FI', 'motorcycle_Yamaha_Fazer', 'motorcycle_Yamaha_Libero', 'motorcycle_Yamaha_R15', 'motorcycle_Yamaha_RX100', 'scooter_Bajaj_Chetak', 'scooter_Hero_Duet', 'scooter_Hero_Maestro', 'scooter_Hero_Pleasure', 'scooter_Honda_Activa', 'scooter_Honda_Aviator', 'scooter_Honda_Dio', 'scooter_Honda_Grazia', 'scooter_Mahindra_Gusto', 'scooter_Others', 'scooter_Suzuki_Access', 'scooter_Suzuki_Burgman', 'scooter_Suzuki_Swish', 'scooter_TVS_Jupiter', 'scooter_TVS_Ntorq', 'scooter_TVS_Pep', 'scooter_TVS_Streak', 'scooter_TVS_Wego', 'scooter_TVS_Zest', 'scooter_Vespa_VXL125', 'scooter_Yamaha_Fascino125', 'scooter_Yamaha_RayZR', 'truck_AshokLeyland', 'truck_BharatBenz', 'truck_Eicher', 'truck_Mahindra', 'truck_Others', 'truck_SML', 'truck_Tata']
l2_names_order = []
l1_names_order = []
l3_l2_mapper = []
l3_l1_mapper = []
pname = ''
p1name = ''
for l3i in l3_class_indexes_names:
    if(pname!='_'.join(l3i.split('_')[:2])):
        pname = '_'.join(l3i.split('_')[:2])
        l2_names_order.append(pname)
    l3_l2_mapper.append(len(l2_names_order)-1)
    
    if(p1name!=l3i.split('_')[0]):
        p1name = l3i.split('_')[0]
        l1_names_order.append(p1name)
    l3_l1_mapper.append(len(l1_names_order)-1)
    
l3_class_indexes_names = {k: v for k, v in enumerate(l3_class_indexes_names)}


tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=l3_class_indexes_names)
p.mean(), r.mean(), ap[:, 0].mean()

# f-score wala best i ka value: 461 out of  1000
# (0.014658523305032133, 0.8166648909026482, 0.3274881928637568)

(0.5229404179827262, 0.30558456010527546, 0.33032474884119983)

In [4]:
l2_names_dict = {}
for li, ci in enumerate(l2_names_order):
    l2_names_dict[li] = ci
l1_names_dict = {}
for li, ci in enumerate(l1_names_order):
    l1_names_dict[li] = ci

In [6]:
for ki in range(stats[2].shape[0]):
    stats[2][ki] = l3_l1_mapper[int(stats[2][ki])] 
for ki in range(stats[3].shape[0]):
    stats[3][ki] = l3_l1_mapper[int(stats[3][ki])]
tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=l1_names_dict)
p.mean(), r.mean(), ap[:, 0].mean()


(0.5961094681084244, 0.5193551368869126, 0.5441993517410303)

In [6]:
for ki in range(stats[2].shape[0]):
    stats[2][ki] = l3_l2_mapper[int(stats[2][ki])]
for ki in range(stats[3].shape[0]):
    stats[3][ki] = l3_l2_mapper[int(stats[3][ki])]
tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=l2_names_dict)
p.mean(), r.mean(), ap[:, 0].mean()


(0.6598387112896565, 0.3687329039041143, 0.4146279882894984)

In [10]:
import pandas as pd, numpy as np
kdf = pd.read_csv("test_dumps_orig_list2_outl3.txt", sep='\t',header=None)
uniqus = np.unique(kdf[1].to_list())
print(len(uniqus), uniqus)
kdf.head()

175 [  0   1   2   3   4   5   6   7   9  10  12  13  14  17  18  20  22  23
  24  25  26  27  29  31  32  33  34  35  37  38  39  41  42  43  44  45
  46  47  50  51  52  53  54  55  56  57  58  60  61  62  63  64  65  66
  67  68  69  70  71  72  73  74  75  76  77  78  81  85  86  87  88  89
  90  91  93  94  95  96  98  99 100 103 104 106 107 108 109 110 111 112
 114 115 116 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 153 154 155 156 157 158 160 161 165 166 167 168 171 173 174 175 176 177
 178 179 180 181 182 183 184 185 186 187 189 190 191 192 193 196 197 198
 200 202 204 205 206 208 209 210 211 212 213 215 216]


,0,1,2
0,mainbatchi0_imgbatchi0_predi0_0.png,2,12
1,mainbatchi0_imgbatchi0_predi1_1.png,111,12
2,mainbatchi0_imgbatchi0_predi2_2.png,106,12
3,mainbatchi0_imgbatchi0_predi3_3.png,157,12
4,mainbatchi0_imgbatchi0_predi4_4.png,2,12


In [18]:
l2pred_labels = [l3_l2_mapper[ui] for ui in kdf[1].to_list()]
print(np.unique(l2pred_labels).shape)
pd.DataFrame({0:kdf[0], 1:l2pred_labels, 2:kdf[2]}).to_csv("test_dumps_orig_list2_outl2_new.txt",sep='\t',header = False, index=False)

(51,)


In [19]:
l1pred_labels = [l3_l1_mapper[ui] for ui in kdf[1].to_list()]
print(np.unique(l1pred_labels).shape)
pd.DataFrame({0:kdf[0], 1:l1pred_labels, 2:kdf[2]}).to_csv("test_dumps_orig_list2_outl1_new.txt",sep='\t',header = False, index=False)

(7,)


In [11]:
l3_class_indexes = {'autorickshaw_Atul_Atul': 1, 'autorickshaw_autorickshaw_autorickshaw': 2, 'autorickshaw_Bajaj_Bajaj': 3, 'autorickshaw_Mahindra_Mahindra': 4, 'autorickshaw_Piaggio_Piaggio': 5, 'autorickshaw_TVS_TVS': 6, 'bus_bus_bus': 7, 'car_Audi_A3': 8, 'car_Audi_Q3': 9, 'car_Bmw_2-Series-220d': 10, 'car_Bmw_3-Series': 11, 'car_Bmw_X1': 12, 'car_car_car': 13, 'car_Chevrolet_Aveo': 14, 'car_Chevrolet_Beat': 15, 'car_Chevrolet_Cruze': 16, 'car_Chevrolet_Enjoy': 17, 'car_Chevrolet_Spark': 18, 'car_Chevrolet_Tavera': 19, 'car_Fiat_Linea': 20, 'car_Fiat_PuntoEvo': 21, 'car_Force_TraxToofan': 22, 'car_Ford_Aspire': 23, 'car_Ford_Ecosport': 24, 'car_Ford_EcoSportTitanium': 25, 'car_Ford_Everest': 26, 'car_Ford_Fiesta': 27, 'car_Ford_Figo': 28, 'car_Ford_Ikon': 29, 'car_Hindustan_Ambassador': 30, 'car_Honda_Accord': 31, 'car_Honda_Amaze': 32, 'car_Honda_Brio': 33, 'car_Honda_Brv': 34, 'car_Honda_City': 35, 'car_Honda_Civic': 36, 'car_Honda_Cr-V': 37, 'car_Honda_Jazz': 38, 'car_Honda_Wr-V': 39, 'car_Hyundai_Accent': 40, 'car_Hyundai_Aura': 41, 'car_Hyundai_Creta': 42, 'car_Hyundai_Eon': 43, 'car_Hyundai_I10': 44, 'car_Hyundai_I20': 45, 'car_Hyundai_Santro': 46, 'car_Hyundai_Verna': 47, 'car_Hyundai_Xcent': 48, 'car_Jeep_Compass': 49, 'car_Jeep_Wrangler': 50, 
'car_Mahindra_Bolero': 51, 'car_Mahindra_Reva': 52, 'car_Mahindra_Scorpio': 53, 'car_Mahindra_TUV300': 54, 'car_Mahindra_Verito': 55, 'car_Mahindra_XUV500': 56, 'car_Mahindra_Xylo': 57, 'car_MarutiSuzuki_1000': 58, 'car_MarutiSuzuki_Alto800': 59, 'car_MarutiSuzuki_AltoK10': 60, 'car_MarutiSuzuki_Baleno': 61, 'car_MarutiSuzuki_Celerio': 62, 'car_MarutiSuzuki_Ciaz': 63, 'car_MarutiSuzuki_Dzire': 64, 'car_MarutiSuzuki_Eeco': 65, 'car_MarutiSuzuki_Ertiga': 66, 'car_MarutiSuzuki_Esteem2000': 67, 'car_MarutiSuzuki_Ignis': 68, 'car_MarutiSuzuki_Omni': 69, 'car_MarutiSuzuki_Ritz': 70, 'car_MarutiSuzuki_S-Cross': 71, 'car_MarutiSuzuki_Swift': 72, 'car_MarutiSuzuki_SX4': 73, 'car_MarutiSuzuki_VitaraBrezza': 74, 'car_MarutiSuzuki_WagonR': 75, 'car_MarutiSuzuki_Zen': 76, 'car_Mercedes-Benz_A-Class': 77, 'car_Mercedes-Benz_AmgGt4-DoorCoupe': 78, 'car_Mercedes-Benz_C-Class': 79, 'car_Mercedes-Benz_E-Class': 80, 'car_Mercedes-Benz_G-Class': 81, 'car_Mercedes-Benz_Gla-Class': 82, 'car_Mercedes-Benz_Gls': 83, 'car_Mercedes-Benz_S-Class': 84, 'car_Mitsubishi_Lancer': 85, 'car_Nissan_Micra': 86, 'car_Nissan_Sunny': 87, 'car_Nissan_Terrano': 88, 'car_Renault_Duster': 89, 'car_Renault_Kwid': 90, 'car_Renault_Lodgy': 91, 'car_Renault_Logan': 92, 'car_Renault_Scala': 93, 'car_Skoda_Fabia': 94, 'car_Skoda_Octavia': 95, 'car_Skoda_Rapid': 96, 'car_Skoda_Superb': 97, 'car_TataMotors_Hexa': 98, 'car_TataMotors_Indica': 99, 'car_TataMotors_Indigo': 100,
'car_TataMotors_Nano': 101, 'car_TataMotors_Nexon': 102, 'car_TataMotors_Safari': 103, 'car_TataMotors_Sumo': 104, 'car_TataMotors_Tiago': 105, 'car_TataMotors_Tigor': 106, 'car_TataMotors_Zest': 107, 'car_Toyota_Corolla': 108, 'car_Toyota_Etios': 109, 'car_Toyota_EtiosLiva': 110, 'car_Toyota_Fortuner': 111, 'car_Toyota_Innova': 112, 'car_Toyota_Qualis': 113, 'car_Volkswagen_Ameo': 114, 'car_Volkswagen_Jetta': 115, 'car_Volkswagen_Polo': 116, 'car_Volkswagen_Vento': 117, 'car_Volvo_Xc40': 118, 'car_Volvo_Xc60': 119, 'mini-bus_mini-bus_mini-bus': 120, 'motorcycle_Bajaj_Avenger': 121, 'motorcycle_Bajaj_CT100': 122, 'motorcycle_Bajaj_Discover': 123, 'motorcycle_Bajaj_Discover100': 124, 'motorcycle_Bajaj_Discover110': 125, 'motorcycle_Bajaj_Discover125': 126, 'motorcycle_Bajaj_Discover135': 127, 'motorcycle_Bajaj_Platina': 128, 'motorcycle_Bajaj_Pulsar150': 129, 'motorcycle_Bajaj_Pulsar180': 130, 'motorcycle_Bajaj_Pulsar200': 131, 'motorcycle_Bajaj_Pulsar220F': 132, 'motorcycle_Bajaj_PulsarNS200': 133, 'motorcycle_Bajaj_PulsarRS200': 134, 'motorcycle_Bajaj_V12': 135, 'motorcycle_Bajaj_V15': 136, 'motorcycle_Hero_Glamour': 137, 'motorcycle_Hero_HFDeluxe': 138, 'motorcycle_Hero_Hunk': 139, 'motorcycle_Hero_Passion': 140, 'motorcycle_Hero_PassionPlus': 141, 'motorcycle_Hero_PassionPro': 142, 'motorcycle_Hero_Splendor': 143, 'motorcycle_Hero_SuperSplendor': 144, 'motorcycle_Hero_XPulse200': 145, 'motorcycle_HeroHonda_CBZ': 146, 'motorcycle_HeroHonda_SplendorNXG': 147, 'motorcycle_Honda_CBHornet160R': 148, 'motorcycle_Honda_CBTwister': 149, 'motorcycle_Honda_Karizma': 150, 'motorcycle_Honda_KarizmaZMR': 151, 'motorcycle_Honda_Shine': 152, 'motorcycle_Honda_SP125': 153, 'motorcycle_Honda_StunnerCBF': 154, 'motorcycle_Honda_Unicorn': 155, 'motorcycle_KTM_Duke': 156, 'motorcycle_Mahindra_Centuro': 157, 'motorcycle_motorcycle_motorcycle': 158, 'motorcycle_RoyalEnfield_Bullet350': 159, 'motorcycle_RoyalEnfield_Bullet500': 160, 'motorcycle_RoyalEnfield_Classic350': 161, 'motorcycle_RoyalEnfield_Classic500': 162, 'motorcycle_RoyalEnfield_ContinentalGT650': 163, 'motorcycle_RoyalEnfield_Interceptor650': 164, 'motorcycle_RoyalEnfield_Meteor350': 165, 'motorcycle_RoyalEnfield_Thunderbird350': 166, 'motorcycle_RoyalEnfield_Thunderbird350X': 167, 'motorcycle_Suzuki_Gixxer': 168, 'motorcycle_Suzuki_Samurai': 169, 'motorcycle_Suzuki_Slingshot': 170, 'motorcycle_TVS_ApacheRR310': 171, 'motorcycle_TVS_ApacheRTR160': 172, 'motorcycle_TVS_ApacheRTR200': 173, 'motorcycle_TVS_Excel100': 174, 'motorcycle_TVS_ExcelHeavyDuty': 175, 'motorcycle_TVS_Sport': 176, 'motorcycle_TVS_StarCityPlus': 177, 'motorcycle_TVS_Victor': 178, 'motorcycle_TVS_XL100': 179, 'motorcycle_Yamaha_Crux': 180, 'motorcycle_Yamaha_Fazer': 181, 'motorcycle_Yamaha_FZ25': 182, 'motorcycle_Yamaha_FZS-FI': 183, 'motorcycle_Yamaha_FZ-V3': 184, 'motorcycle_Yamaha_Libero': 185, 'motorcycle_Yamaha_R15': 186, 'motorcycle_Yamaha_RX100': 187, 'scooter_Bajaj_Chetak': 188, 'scooter_Hero_Duet': 189, 'scooter_Hero_Maestro': 190, 'scooter_Hero_Pleasure': 191, 'scooter_Honda_Activa': 192, 'scooter_Honda_Aviator': 193, 'scooter_Honda_Dio': 194, 'scooter_Honda_Grazia': 195, 'scooter_Mahindra_Gusto': 196, 'scooter_scooter_scooter': 197, 'scooter_Suzuki_Access': 198, 'scooter_Suzuki_Burgman': 199, 'scooter_Suzuki_Swish': 200, 'scooter_TVS_Jupiter': 201, 'scooter_TVS_Ntorq': 202, 'scooter_TVS_Pep': 203, 'scooter_TVS_Streak': 204,
'scooter_TVS_Wego': 205, 'scooter_TVS_Zest': 206, 'scooter_Vespa_VXL125': 207, 'scooter_Vespa_ZX125': 208, 'scooter_Yamaha_Fascino125': 209, 'scooter_Yamaha_RayZR': 210, 'truck_AshokLeyland_AshokLeyland': 211, 'truck_BharatBenz_BharatBenz': 212, 'truck_Eicher_Eicher': 213, 'truck_Mahindra_Mahindra': 214, 'truck_SML_SML': 215, 'truck_Tata_Tata': 216, 'truck_truck_truck': 217}
l3_class_indexes_names = list(l3_class_indexes.keys())
l2_names_order = []
l1_names_order = []
l3_l2_mapper = []
l3_l1_mapper = []
pname = ''
p1name = ''
for l3i in l3_class_indexes_names:
    if(pname!='_'.join(l3i.split('_')[:2])):
        pname = '_'.join(l3i.split('_')[:2])
        l2_names_order.append(pname)
    l3_l2_mapper.append(len(l2_names_order)-1)
    
#     if(pname!=l3i.split('_')[1]):
#         pname = l3i.split('_')[1]
#         l2_names_order.append(pname)
#     l3_l2_mapper.append(len(l2_names_order)-1)
    
    if(p1name!=l3i.split('_')[0]):
        p1name = l3i.split('_')[0]
        l1_names_order.append(p1name)
    l3_l1_mapper.append(len(l1_names_order)-1)
print("length of l2 unqiue names: ", len(l2_names_order))
print("length of l1 unqiue names: ", len(l1_names_order))


length of l2 unqiue names:  57
length of l1 unqiue names:  7


In [12]:
np.unique(l2_names_order).shape

(57,)

In [1]:
"""
Validate a trained YOLOv5 model accuracy on a custom dataset

Usage:
    $ python3 val.py --workers 4 --batch-size 4 --weights yolov5x.pt --data IDD_FGD_L3.yaml --img 640 --task test --name IDD_FGD_L3_test
    $ python3 val.py --workers 4 --batch-size 4 --weights runs/train/idd_fgd_l3/best_e16.pt --data IDD_FGD_L3.yaml --task test --name IDD_FGD_L3_test --device 1 --iou-thres 0.5

args: data=IDD_FGD_L3.yaml, weights=['runs/train/idd_fgd_l3/best_e16.pt'], batch_size=4, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=4, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=IDD_FGD_L3_test, exist_ok=False, half=False, dnn=False

output:
test: Scanning '/home/chirag/cp_wks/datasets/IDD_FGD_L3/test/labels' images and labels...1083 found, 0 missing, 0 empty, 0 corrupt: 100%|██████████| 1083/1083 [00:00<00:00, 11573.25it/s]                 
test: New cache created: /home/chirag/cp_wks/datasets/IDD_FGD_L3/test/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 271/271 [05:00<00:00,  1.11s/it]                                                                  
                 all       1083       4891    0.00785      0.661     0.0229     0.0185
Speed: 0.5ms pre-process, 259.7ms inference, 14.1ms NMS per image at shape (4, 3, 640, 640)
Results saved to runs/val/IDD_FGD_L3_test2

"""

import argparse
import json
import os
import sys
from pathlib import Path

import numpy as np
import torch
from tqdm import tqdm
import pandas as pd
# pdffk = pd.read_csv("test_dumps_list2_out.txt",sep='\t',header=None)
pdffk = pd.read_csv("test_dumps_orig_list2_outl3.txt",sep='\t',header=None)
l3_new_index_mapper = list(pd.read_csv("l3_new_index_mapper.csv")['l3_new_index'])
fnamespdf = list(pdffk[0])

from models.common import DetectMultiBackend
from utils.callbacks import Callbacks
from utils.dataloaders import create_dataloader
from utils.general import (LOGGER, check_dataset, check_img_size, check_requirements, check_yaml,
                           coco80_to_coco91_class, colorstr, emojis, increment_path, non_max_suppression, print_args,
                           scale_coords, xywh2xyxy, xyxy2xywh)
from utils.metrics import ConfusionMatrix, ap_per_class, box_iou
from utils.plots import output_to_target, plot_images, plot_val_study
from utils.torch_utils import select_device, time_sync

def save_one_txt(predn, save_conf, shape, file):
    # Save one txt result
    gn = torch.tensor(shape)[[1, 0, 1, 0]]  # normalization gain whwh
    for *xyxy, conf, cls in predn.tolist():
        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
        with open(file, 'a') as f:
            f.write(('%g ' * len(line)).rstrip() % line + '\n')


def save_one_json(predn, jdict, path, class_map):
    # Save one JSON result {"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}
    image_id = int(path.stem) if path.stem.isnumeric() else path.stem
    box = xyxy2xywh(predn[:, :4])  # xywh
    box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
    for p, b in zip(predn.tolist(), box.tolist()):
        jdict.append({
            'image_id': image_id,
            'category_id': class_map[int(p[5])],
            'bbox': [round(x, 3) for x in b],
            'score': round(p[4], 5)})


def process_batch(detections, labels, iouv):
    """
    Return correct predictions matrix. Both sets of boxes are in (x1, y1, x2, y2) format.
    Arguments:
        detections (Array[N, 6]), x1, y1, x2, y2, conf, class
        labels (Array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (Array[N, 10]), for 10 IoU levels
    """
    correct = np.zeros((detections.shape[0], iouv.shape[0])).astype(bool)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    correct_class = labels[:, 0:1] == detections[:, 5]
    matches_iou0 = np.array([])
    for i in range(len(iouv)):
        x = torch.where((iou >= iouv[i]) & correct_class)  # IoU > threshold and classes match
        if x[0].shape[0]:
            matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
            if x[0].shape[0] > 1:
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                # matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
            correct[matches[:, 1].astype(int), i] = True
            if(i==0):
                matches_iou0 = matches.copy()
    return torch.tensor(correct, dtype=torch.bool, device=iouv.device), matches_iou0



In [2]:

parser = argparse.ArgumentParser()
parser.add_argument('--data', type=str, default='data/coco128.yaml', help='dataset.yaml path')
parser.add_argument('--weights', nargs='+', type=str, default= 'yolov5s.pt', help='model.pt path(s)')
parser.add_argument('--batch-size', type=int, default=32, help='batch size')
parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='inference size (pixels)')
parser.add_argument('--conf-thres', type=float, default=0.001, help='confidence threshold')
parser.add_argument('--iou-thres', type=float, default=0.5, help='NMS IoU threshold')
parser.add_argument('--task', default='val', help='train, val, test, speed or study')
parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
parser.add_argument('--workers', type=int, default=8, help='max dataloader workers (per RANK in DDP mode)')
parser.add_argument('--single-cls', action='store_true', help='treat as single-class dataset')
parser.add_argument('--augment', action='store_true', help='augmented inference')
parser.add_argument('--verbose', action='store_true', help='report mAP by class')
parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
parser.add_argument('--save-hybrid', action='store_true', help='save label+prediction hybrid results to *.txt')
parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
parser.add_argument('--save-json', action='store_true', help='save a COCO-JSON results file')
parser.add_argument('--project', default='runs/val', help='save to project/name')
parser.add_argument('--name', default='exp', help='save to project/name')
parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
opt = parser.parse_args("")
opt.workers = 4
opt.batch_size = 4
# opt.weights = "runs/train/idd_fgd_l3/best_e16.pt"
# opt.weights = "runs/train/idd_fgd_l1.pt"
opt.weights = "runs/train/exp2/weights/best.pt"
opt.data = "IDD_FGD_L1.yaml"
opt.task = "test"
opt.name = "IDD_FGD_L1_test"

opt.data = check_yaml(opt.data)  # check YAML
opt.save_json |= opt.data.endswith('coco.yaml')
opt.save_txt |= opt.save_hybrid

vars(opt)


{'data': 'IDD_FGD_L1.yaml',
 'weights': 'runs/train/exp2/weights/best.pt',
 'batch_size': 4,
 'imgsz': 640,
 'conf_thres': 0.001,
 'iou_thres': 0.5,
 'task': 'test',
 'device': '',
 'workers': 4,
 'single_cls': False,
 'augment': False,
 'verbose': False,
 'save_txt': False,
 'save_hybrid': False,
 'save_conf': False,
 'save_json': False,
 'project': 'runs/val',
 'name': 'IDD_FGD_L1_test',
 'exist_ok': False,
 'half': False,
 'dnn': False}

In [3]:
# @torch.no_grad()
data = opt.data
weights=opt.weights
batch_size=opt.batch_size
imgsz=640
conf_thres=0.001  # confidence threshold
iou_thres=0.5  # NMS IoU threshold
task='test'  # train, val, test, speed or study
device=''  # cuda device, i.e. 0 or 0,1,2,3 or cpu
workers=opt.workers  # max dataloader workers (per RANK in DDP mode)
single_cls=False  # treat as single-class dataset
augment=False  # augmented inference
verbose=False  # verbose output
save_txt=False  # save results to *.txt
save_hybrid=False  # save label+prediction hybrid results to *.txt
save_conf=False  # save confidences in --save-txt labels
save_json=False  # save a COCO-JSON results file
project=opt.project  # save to project/name
name=opt.name  # save to project/name
exist_ok=False,  # existing project/name ok, do not increment
half=opt.half  # use FP16 half-precision inference
dnn=False  # use OpenCV DNN for ONNX inference
model=None
dataloader=None
save_dir=Path('')
plots=True
callbacks=Callbacks()
compute_loss=None

if(not os.path.exists('/scratch/prafful/test_dumps')):
    os.makedirs('/scratch/prafful/test_dumps')
# Initialize/load model and set device
training = model is not None
if training:  # called by train.py
    device, pt, jit, engine = next(model.parameters()).device, True, False, False  # get model device, PyTorch model
    half &= device.type != 'cpu'  # half precision only supported on CUDA
    model.half() if half else model.float()
else:  # called directly
    device = select_device(device, batch_size=batch_size)

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, pt, jit, engine = model.stride, model.pt, model.jit, model.engine
    imgsz = check_img_size(imgsz, s=stride)  # check image size
    half = model.fp16  # FP16 supported on limited backends with CUDA
    if engine:
        batch_size = model.batch_size
    else:
        device = model.device
        if not (pt or jit):
            batch_size = 1  # export.py models default to batch-size 1
            LOGGER.info(f'Forcing --batch-size 1 square inference (1,3,{imgsz},{imgsz}) for non-PyTorch models')

    # Data
    data = check_dataset(data)  # check

# Configure
model.eval()
cuda = device.type != 'cpu'
is_coco = isinstance(data.get('val'), str) and data['val'].endswith(f'coco{os.sep}val2017.txt')  # COCO dataset
nc = 1 if single_cls else int(data['nc'])  # number of classes
iouv = torch.linspace(0.5, 0.95, 10, device=device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

# Dataloader
if not training:
    if pt and not single_cls:  # check --weights are trained on --data
        ncm = model.model.nc
        assert ncm == nc, f'{weights} ({ncm} classes) trained on different --data than what you passed ({nc} ' \
                          f'classes). Pass correct combination of --weights and --data that are trained together.'
    model.warmup(imgsz=(1 if pt else batch_size, 3, imgsz, imgsz))  # warmup
    pad = 0.0 if task in ('speed', 'benchmark') else 0.5
    rect = False if task == 'benchmark' else pt  # square inference for benchmarks
    task = task if task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
    dataloader = create_dataloader(data[task],
                                   imgsz,
                                   batch_size,
                                   stride,
                                   single_cls,
                                   pad=pad,
                                   rect=rect,
                                   workers=workers,
                                   prefix=colorstr(f'{task}: '))[0]

seen = 0
confusion_matrix = ConfusionMatrix(nc=nc)
names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
class_map = coco80_to_coco91_class() if is_coco else list(range(1000))
s = ('%20s' + '%11s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
dt, p, r, f1, mp, mr, map50, map = [0.0, 0.0, 0.0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
loss = torch.zeros(3, device=device)
jdict, stats, ap, ap_class = [], [], [], []


YOLOv5 🚀 v6.1-330-ge34ae88 Python-3.7.1 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12054MiB)

Fusing layers... 
Model summary: 367 layers, 46140588 parameters, 0 gradients
test: Scanning '/home2/praffulkumar/cp_wks/datasets/IDD_FGD_L3/test/labels.cache' images and labels... 1083 found, 0 missing, 0 empty, 0 corrupt: 100%|██████████| 1083/1083 [00:00<?, ?it/s]              


In [4]:
callbacks.run('on_val_start')
pbar = tqdm(dataloader, desc=s, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar
ijk=0
lmnop=[]
for batch_i, (im, targets, paths, shapes) in enumerate(pbar):
    callbacks.run('on_val_batch_start')
    ### loading l3_labels
    targets_l3 = targets.clone()
    for ip, pi in enumerate(paths):
        pi_l3_labels = list(pd.read_csv(pi.replace('_L1/','_L3/').replace('images','labels_l3_backup')[:-4]+'.txt',sep=' ',header=None)[0])
        for ti, pil3 in zip(np.where((targets_l3[:, 0] == ip).cpu().numpy())[0] , pi_l3_labels):
            targets_l3[ti,1] = l3_new_index_mapper[pil3]-1
            
    t1 = time_sync()
    if cuda:
        im = im.to(device, non_blocking=True)
        targets = targets.to(device)
        targets_l3 = targets_l3.to(device)
    im = im.half() if half else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    nb, _, height, width = im.shape  # batch size, channels, height, width
    t2 = time_sync()
    dt[0] += t2 - t1

    # Inference
    out, train_out = model(im) if training else model(im, augment=augment, val=True)  # inference, loss outputs
    dt[1] += time_sync() - t2

    # Loss
    if compute_loss:
        loss += compute_loss([x.float() for x in train_out], targets)[1]  # box, obj, cls

    # NMS
    targets[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
    targets_l3[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
    lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling
    t3 = time_sync()
    out = non_max_suppression(out, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=single_cls)
    dt[2] += time_sync() - t3

    break

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0%|          | 0/271 [00:00<?, ?it/s]                                                                            


In [5]:
# Metrics
for si, pred in enumerate(out):
    labels = targets[targets[:, 0] == si, 1:]
    labels_l3 = targets_l3[targets_l3[:, 0] == si, 1:]

    nl, npr = labels.shape[0], pred.shape[0]  # number of labels, predictions
    path, shape = Path(paths[si]), shapes[si][0]
    correct = torch.zeros(npr, niou, dtype=torch.bool, device=device)  # init
    seen += 1

    if npr == 0:
        if nl:
            stats.append((correct, *torch.zeros((2, 0), device=device), labels[:, 0]))
#                     if plots:
#                         confusion_matrix.process_batch(detections=None, labels=labels[:, 0])
        continue

    # Predictions
    if single_cls:
        pred[:, 5] = 0
    predn = pred.clone()
    scale_coords(im[si].shape[1:], predn[:, :4], shape, shapes[si][1])  # native-space pred

    break

In [25]:
'/'.join('/home2/praffulkumar/cp_wks/datasets/IDD_FGD_L3/test/images/5763.jpg'.split('/')[-3:])

'test/images/5763.jpg'

In [16]:
# import pandas as pd
# pd.DataFrame(pred[:, :4].cpu().numpy(),columns=['x1','y1','x2','y2'])

In [6]:
### crop image or grab the HRN l3 output
predi = pred[:, :4].cpu().float().numpy()
imik = im[si].cpu().float().numpy()
imik = imik.transpose(1, 2, 0)
if np.max(imik) <= 1:
    imik *= 255
for pis in range(predi.shape[0]):
    cv2.imwrite("/scratch/prafful/test_dumps_orig/mainbatchi"+str(batch_i)+"_imgbatchi"+str(si)+"_predi"+str(pis)+"_"+str(ijk)+".png",imik[int(predi[pis,1]):int(predi[pis,3]),int(predi[pis,0]):int(predi[pis,2])].astype('uint8')[:,:,::-1])
    try:
        pred[pis, 5] = pdffk[1][fnamespdf.index("mainbatchi"+str(batch_i)+"_imgbatchi"+str(si)+"_predi"+str(pis)+"_"+str(ijk)+".png")]
        predn[pis, 5] = pred[pis, 5]
    except:
        lmnop.append(1)
#         print("bla")
        
    cv2.imwrite("/scratch/prafful/test_dumps/mainbatchi"+str(batch_i)+"_imgbatchi"+str(si)+"_predi"+str(pis)+"_"+str(ijk)+".png",imik[int(predi[pis,1]):int(predi[pis,3]),int(predi[pis,0]):int(predi[pis,2])].astype('uint8')[:,:,::-1])
    ijk+=1


In [9]:
pred[:, 5]

tensor([  2., 111.,  63., 157.,   2.,   2.,   2., 157.,   2.,   2.,   2., 111., 111., 111., 111., 111., 111.,  63.,  63.,  63.,  63.,  63.,   6.,  98.,  63.], device='cuda:0')

In [8]:
targets[targets[:, 0] == si, 1:]

tensor([[  1.00000, 315.51068, 236.87500,  67.29828,  69.14075],
        [  1.00000, 383.30869, 229.56204,  63.63348,  50.52592],
        [  3.00000, 224.55804, 275.76666, 127.26703, 150.24815]], device='cuda:0')

In [10]:
targets_l3[targets_l3[:, 0] == si, 1:]

tensor([[1.12000e+02, 4.69510e-01, 6.16862e-01, 1.00146e-01, 1.80054e-01],
        [1.07000e+02, 5.70400e-01, 5.97818e-01, 9.46927e-02, 1.31578e-01],
        [3.00000e+00, 3.34164e-01, 7.18142e-01, 1.89385e-01, 3.91271e-01]], device='cuda:0')

In [6]:
# Evaluate
if nl:
    tbox = xywh2xyxy(labels[:, 1:5])  # target boxes
    scale_coords(im[si].shape[1:], tbox, shape, shapes[si][1])  # native-space labels
    labelsn = torch.cat((labels[:, 0:1], tbox), 1)  # native-space labels
    correct, correct_matches_iou0 = process_batch(predn, labelsn, iouv)

In [8]:
correct.shape,torch.sum(correct[:,0]), predn.shape, labelsn.shape, iouv.shape

(torch.Size([25, 10]),
 tensor(3),
 torch.Size([25, 6]),
 torch.Size([3, 5]),
 torch.Size([10]))

In [7]:
predn[correct[:,0],5],labelsn[:,0],predn[:,5]

(tensor([3., 1., 1.]),
 tensor([1., 1., 3.]),
 tensor([3., 1., 1., 2., 1., 4., 0., 0., 2., 5., 6., 3., 2., 6., 5., 0., 4., 5., 3., 2., 4., 0., 0., 1., 6.]))

In [13]:
if(correct_matches_iou0.shape[0]>0):
    correct_matches_iou0

In [17]:
correct = np.zeros((predn.shape[0], iouv.shape[0])).astype(bool)
iou = box_iou(labelsn[:, 1:], predn[:, :4])
correct_class = labelsn[:, 0:1] == predn[:, 5]
iou.shape,correct_class.shape,iou,correct_class

(torch.Size([3, 25]),
 torch.Size([3, 25]),
 tensor([[0.02463, 0.92835, 0.00000, 0.00000, 0.02480, 0.02551, 0.02551, 0.00000, 0.02551, 0.02551, 0.02551, 0.95488, 0.94834, 0.95488, 0.94834, 0.95488, 0.95488, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.10178, 0.00000],
         [0.00000, 0.00000, 0.89970, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.90273, 0.90273, 0.90829, 0.89760, 0.89970, 0.00000, 0.29116, 0.90273],
         [0.97396, 0.02159, 0.00000, 0.00000, 0.97178, 0.97388, 0.97388, 0.00000, 0.97388, 0.97388, 0.97388, 0.01908, 0.01904, 0.01908, 0.01904, 0.01908, 0.01908, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000]]),
 tensor([[False,  True,  True, False,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False,  True, False],
         [False,  True,  True, False,  True, False, False, F

In [16]:
i=0
x = torch.where((iou >= iouv[i]) & correct_class)  # IoU > threshold and classes match
x

(tensor([0, 1, 2]), tensor([1, 2, 0]))

In [25]:
matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
matches = matches[matches[:, 2].argsort()[::-1]]
matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
matches

array([[          0,           1,     0.92835],
       [          1,           2,      0.8997],
       [          2,           0,     0.97396]], dtype=float32)

In [ ]:
if x[0].shape[0]:
    matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
    if x[0].shape[0] > 1:
        matches = matches[matches[:, 2].argsort()[::-1]]
        matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
        # matches = matches[matches[:, 2].argsort()[::-1]]
        matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
    correct[matches[:, 1].astype(int), i] = True

In [18]:
np.sum(correct.cpu().numpy()[:,0]),correct.shape,predn.shape,labelsn.shape

(3, torch.Size([25, 10]), torch.Size([25, 6]), torch.Size([3, 5]))

In [9]:
im[si].shape[1:], tbox, shape, shapes[si][1]

(torch.Size([384, 672]),
 tensor([[ 799.80634,  571.00000, 1002.26355,  779.00000],
         [1009.27942,  577.00000, 1200.71155,  729.00006],
         [ 435.98465,  565.99994,  818.84924, 1018.00000]], device='cuda:0'),
 [1080, 1921],
 [[0.33240740740740743, 0.3331598125976054], [16.0, 12.5]])

In [8]:
labelsn

tensor([[1.00000e+00, 7.99806e+02, 5.71000e+02, 1.00226e+03, 7.79000e+02],
        [1.00000e+00, 1.00928e+03, 5.77000e+02, 1.20071e+03, 7.29000e+02],
        [3.00000e+00, 4.35985e+02, 5.66000e+02, 8.18849e+02, 1.01800e+03]], device='cuda:0')

In [20]:
import pickle
with open('/home2/praffulkumar/cp_wks/yolov5/v5l_fgd_l1_test_stats.pkl', 'rb') as file:
# with open('/home2/praffulkumar/cp_wks/yolov5/v5l_fgd_l3_test_stats.pkl', 'rb') as file:
    # A new file will be created
    stats = pickle.load(file)

# stats = (correct, conf, pcls, tcls)
print(stats[0].shape, stats[1].shape, stats[2].shape, stats[3].shape )

(39650, 10) (39650,) (39650,) (4891,)


In [21]:
import numpy as np
aka = stats[2][stats[0][:,0]]
np.min(aka),np.max(aka),np.unique(aka).shape,aka.shape,np.unique(stats[3]).shape

(0.0, 6.0, (7,), (4642,), (7,))

In [3]:
import numpy as np
aka = stats3[2][stats3[0][:,0]]
np.min(aka),np.max(aka),np.unique(aka).shape,aka.shape,np.unique(stats3[3]).shape

(1.0, 216.0, (109,), (3609,), (170,))

In [5]:
np.sum(np.multiply(stats[0][:,0], stats3[0][:,0]==False)), 4642-3609

(1178, 1033)

In [1]:
import pickle
with open('/home2/praffulkumar/cp_wks/yolov5/v5l_fgd_l1_test_stats.pkl', 'rb') as file:
    # A new file will be created
    stats = pickle.load(file)

with open('/home2/praffulkumar/cp_wks/yolov5/v5l_fgd_l3_test_stats.pkl', 'rb') as file3:
    # A new file will be created
    stats3 = pickle.load(file3)

# # stats = (correct, conf, pcls, tcls)
# stats[2] = stats3[2]
# stats[3] = stats3[3]
print(stats[0].shape, stats[1].shape, stats[2].shape, stats[3].shape )

(39650, 10) (39650,) (39650,) (4891,)


In [2]:
import pandas as pd
imgnames = pd.read_csv("test_dumps_orig_list2.txt",sep='\t',header=None)[0]
img_idxs = [int(imi.split('_')[-1][:-4]) for imi in imgnames]

In [3]:
l3gt_pdf = pd.read_csv("l3gt_foryoloobjects.csv")
yoloobject_img_names = list(l3gt_pdf['yoloobject_img_name'])
l3gt_pdf.shape,l3gt_pdf.head()

((4642, 3),
    Unnamed: 0            yoloobject_img_name  l3gt_foryoloobject
 0           0  mainbatchi0_imgbatchi0_predi1                 111
 1           1  mainbatchi0_imgbatchi0_predi2                 106
 2           2  mainbatchi0_imgbatchi0_predi0                   2
 3           3  mainbatchi0_imgbatchi1_predi0                 106
 4           4  mainbatchi0_imgbatchi2_predi2                  31)

In [11]:
l3gt_objnames = []
for l3ii in l3gt_pdf['l3gt_foryoloobject']:
    l3gt_objnames.append(l3_class_indexes_names[l3ii])
pd.DataFrame({0:[ymi+'.png' for ymi in yoloobject_img_names], 1:l3gt_objnames}).to_csv("orig_gt_object_images_list2.txt",sep='\t',header=False,index=False)
pd.read_csv("orig_gt_object_images_list2.txt",sep='\t',header=None).head()

,0,1
0,mainbatchi0_imgbatchi0_predi1.png,car_Toyota_Innova
1,mainbatchi0_imgbatchi0_predi2.png,car_TataMotors_Zest
2,mainbatchi0_imgbatchi0_predi0.png,autorickshaw_Bajaj_Bajaj
3,mainbatchi0_imgbatchi1_predi0.png,car_TataMotors_Zest
4,mainbatchi0_imgbatchi2_predi2.png,car_Honda_Amaze


In [4]:
import os, pandas as pd
l3gt_pdf = pd.read_csv("/home2/praffulkumar/cp_wks/yolov5/orig_gt_object_images_list2.txt",sep='\t',header=None)
yoloobject_img_names = [imi[:-4] for imi in list(l3gt_pdf[0])]

imgnames = pd.read_csv("/home2/praffulkumar/cp_wks/yolov5/test_dumps_orig_list2.txt",sep='\t',header=None)[0]
imgnames_clipped = ['_'.join(imi.split('_')[:3]) for imi in imgnames]
pdffk = pd.read_csv("/home2/praffulkumar/cp_wks/yolov5/test_dumps_orig_list2_out.txt",sep='\t',header=None)

l3_class_indexes = {'autorickshaw_Atul_Atul': 1, 'autorickshaw_autorickshaw_autorickshaw': 2, 'autorickshaw_Bajaj_Bajaj': 3, 'autorickshaw_Mahindra_Mahindra': 4, 'autorickshaw_Piaggio_Piaggio': 5, 'autorickshaw_TVS_TVS': 6, 'bus_bus_bus': 7, 'car_Audi_A3': 8, 'car_Audi_Q3': 9, 'car_Bmw_2-Series-220d': 10, 'car_Bmw_3-Series': 11, 'car_Bmw_X1': 12, 'car_car_car': 13, 'car_Chevrolet_Aveo': 14, 'car_Chevrolet_Beat': 15, 'car_Chevrolet_Cruze': 16, 'car_Chevrolet_Enjoy': 17, 'car_Chevrolet_Spark': 18, 'car_Chevrolet_Tavera': 19, 'car_Fiat_Linea': 20, 'car_Fiat_PuntoEvo': 21, 'car_Force_TraxToofan': 22, 'car_Ford_Aspire': 23, 'car_Ford_Ecosport': 24, 'car_Ford_EcoSportTitanium': 25, 'car_Ford_Everest': 26, 'car_Ford_Fiesta': 27, 'car_Ford_Figo': 28, 'car_Ford_Ikon': 29, 'car_Hindustan_Ambassador': 30, 'car_Honda_Accord': 31, 'car_Honda_Amaze': 32, 'car_Honda_Brio': 33, 'car_Honda_Brv': 34, 'car_Honda_City': 35, 'car_Honda_Civic': 36, 'car_Honda_Cr-V': 37, 'car_Honda_Jazz': 38, 'car_Honda_Wr-V': 39, 'car_Hyundai_Accent': 40, 'car_Hyundai_Aura': 41, 'car_Hyundai_Creta': 42, 'car_Hyundai_Eon': 43, 'car_Hyundai_I10': 44, 'car_Hyundai_I20': 45, 'car_Hyundai_Santro': 46, 'car_Hyundai_Verna': 47, 'car_Hyundai_Xcent': 48, 'car_Jeep_Compass': 49, 'car_Jeep_Wrangler': 50, 'car_Mahindra_Bolero': 51, 'car_Mahindra_Reva': 52, 'car_Mahindra_Scorpio': 53, 'car_Mahindra_TUV300': 54, 'car_Mahindra_Verito': 55, 'car_Mahindra_XUV500': 56, 'car_Mahindra_Xylo': 57, 'car_MarutiSuzuki_1000': 58, 'car_MarutiSuzuki_Alto800': 59, 'car_MarutiSuzuki_AltoK10': 60, 'car_MarutiSuzuki_Baleno': 61, 'car_MarutiSuzuki_Celerio': 62, 'car_MarutiSuzuki_Ciaz': 63, 'car_MarutiSuzuki_Dzire': 64, 'car_MarutiSuzuki_Eeco': 65, 'car_MarutiSuzuki_Ertiga': 66, 'car_MarutiSuzuki_Esteem2000': 67, 'car_MarutiSuzuki_Ignis': 68, 'car_MarutiSuzuki_Omni': 69, 'car_MarutiSuzuki_Ritz': 70, 'car_MarutiSuzuki_S-Cross': 71, 'car_MarutiSuzuki_Swift': 72, 'car_MarutiSuzuki_SX4': 73, 'car_MarutiSuzuki_VitaraBrezza': 74, 'car_MarutiSuzuki_WagonR': 75, 'car_MarutiSuzuki_Zen': 76, 'car_Mercedes-Benz_A-Class': 77, 'car_Mercedes-Benz_AmgGt4-DoorCoupe': 78, 'car_Mercedes-Benz_C-Class': 79, 'car_Mercedes-Benz_E-Class': 80, 'car_Mercedes-Benz_G-Class': 81, 'car_Mercedes-Benz_Gla-Class': 82, 'car_Mercedes-Benz_Gls': 83, 'car_Mercedes-Benz_S-Class': 84, 'car_Mitsubishi_Lancer': 85, 'car_Nissan_Micra': 86, 'car_Nissan_Sunny': 87, 'car_Nissan_Terrano': 88, 'car_Renault_Duster': 89, 'car_Renault_Kwid': 90, 'car_Renault_Lodgy': 91, 'car_Renault_Logan': 92, 'car_Renault_Scala': 93, 'car_Skoda_Fabia': 94, 'car_Skoda_Octavia': 95, 'car_Skoda_Rapid': 96, 'car_Skoda_Superb': 97, 'car_TataMotors_Hexa': 98, 'car_TataMotors_Indica': 99, 'car_TataMotors_Indigo': 100,'car_TataMotors_Nano': 101, 'car_TataMotors_Nexon': 102, 'car_TataMotors_Safari': 103, 'car_TataMotors_Sumo': 104, 'car_TataMotors_Tiago': 105, 'car_TataMotors_Tigor': 106, 'car_TataMotors_Zest': 107, 'car_Toyota_Corolla': 108, 'car_Toyota_Etios': 109, 'car_Toyota_EtiosLiva': 110, 'car_Toyota_Fortuner': 111, 'car_Toyota_Innova': 112, 'car_Toyota_Qualis': 113, 'car_Volkswagen_Ameo': 114, 'car_Volkswagen_Jetta': 115, 'car_Volkswagen_Polo': 116, 'car_Volkswagen_Vento': 117, 'car_Volvo_Xc40': 118, 'car_Volvo_Xc60': 119, 'mini-bus_mini-bus_mini-bus': 120, 'motorcycle_Bajaj_Avenger': 121, 'motorcycle_Bajaj_CT100': 122, 'motorcycle_Bajaj_Discover': 123, 'motorcycle_Bajaj_Discover100': 124, 'motorcycle_Bajaj_Discover110': 125, 'motorcycle_Bajaj_Discover125': 126, 'motorcycle_Bajaj_Discover135': 127, 'motorcycle_Bajaj_Platina': 128, 'motorcycle_Bajaj_Pulsar150': 129, 'motorcycle_Bajaj_Pulsar180': 130, 'motorcycle_Bajaj_Pulsar200': 131, 'motorcycle_Bajaj_Pulsar220F': 132, 'motorcycle_Bajaj_PulsarNS200': 133, 'motorcycle_Bajaj_PulsarRS200': 134, 'motorcycle_Bajaj_V12': 135, 'motorcycle_Bajaj_V15': 136, 'motorcycle_Hero_Glamour': 137, 'motorcycle_Hero_HFDeluxe': 138, 'motorcycle_Hero_Hunk': 139, 'motorcycle_Hero_Passion': 140, 'motorcycle_Hero_PassionPlus': 141, 'motorcycle_Hero_PassionPro': 142, 'motorcycle_Hero_Splendor': 143, 'motorcycle_Hero_SuperSplendor': 144, 'motorcycle_Hero_XPulse200': 145, 'motorcycle_HeroHonda_CBZ': 146, 'motorcycle_HeroHonda_SplendorNXG': 147, 'motorcycle_Honda_CBHornet160R': 148, 'motorcycle_Honda_CBTwister': 149, 'motorcycle_Honda_Karizma': 150, 'motorcycle_Honda_KarizmaZMR': 151, 'motorcycle_Honda_Shine': 152, 'motorcycle_Honda_SP125': 153, 'motorcycle_Honda_StunnerCBF': 154, 'motorcycle_Honda_Unicorn': 155, 'motorcycle_KTM_Duke': 156, 'motorcycle_Mahindra_Centuro': 157, 'motorcycle_motorcycle_motorcycle': 158, 'motorcycle_RoyalEnfield_Bullet350': 159, 'motorcycle_RoyalEnfield_Bullet500': 160, 'motorcycle_RoyalEnfield_Classic350': 161, 'motorcycle_RoyalEnfield_Classic500': 162, 'motorcycle_RoyalEnfield_ContinentalGT650': 163, 'motorcycle_RoyalEnfield_Interceptor650': 164, 'motorcycle_RoyalEnfield_Meteor350': 165, 'motorcycle_RoyalEnfield_Thunderbird350': 166, 'motorcycle_RoyalEnfield_Thunderbird350X': 167, 'motorcycle_Suzuki_Gixxer': 168, 'motorcycle_Suzuki_Samurai': 169, 'motorcycle_Suzuki_Slingshot': 170, 'motorcycle_TVS_ApacheRR310': 171, 'motorcycle_TVS_ApacheRTR160': 172, 'motorcycle_TVS_ApacheRTR200': 173, 'motorcycle_TVS_Excel100': 174, 'motorcycle_TVS_ExcelHeavyDuty': 175, 'motorcycle_TVS_Sport': 176, 'motorcycle_TVS_StarCityPlus': 177, 'motorcycle_TVS_Victor': 178, 'motorcycle_TVS_XL100': 179, 'motorcycle_Yamaha_Crux': 180, 'motorcycle_Yamaha_Fazer': 181, 'motorcycle_Yamaha_FZ25': 182, 'motorcycle_Yamaha_FZS-FI': 183, 'motorcycle_Yamaha_FZ-V3': 184, 'motorcycle_Yamaha_Libero': 185, 'motorcycle_Yamaha_R15': 186, 'motorcycle_Yamaha_RX100': 187, 'scooter_Bajaj_Chetak': 188, 'scooter_Hero_Duet': 189, 'scooter_Hero_Maestro': 190, 'scooter_Hero_Pleasure': 191, 'scooter_Honda_Activa': 192, 'scooter_Honda_Aviator': 193, 'scooter_Honda_Dio': 194, 'scooter_Honda_Grazia': 195, 'scooter_Mahindra_Gusto': 196, 'scooter_scooter_scooter': 197, 'scooter_Suzuki_Access': 198, 'scooter_Suzuki_Burgman': 199, 'scooter_Suzuki_Swish': 200, 'scooter_TVS_Jupiter': 201, 'scooter_TVS_Ntorq': 202, 'scooter_TVS_Pep': 203, 'scooter_TVS_Streak': 204,'scooter_TVS_Wego': 205, 'scooter_TVS_Zest': 206, 'scooter_Vespa_VXL125': 207, 'scooter_Vespa_ZX125': 208, 'scooter_Yamaha_Fascino125': 209, 'scooter_Yamaha_RayZR': 210, 'truck_AshokLeyland_AshokLeyland': 211, 'truck_BharatBenz_BharatBenz': 212, 'truck_Eicher_Eicher': 213, 'truck_Mahindra_Mahindra': 214, 'truck_SML_SML': 215, 'truck_Tata_Tata': 216, 'truck_truck_truck': 217}
l3_class_indexes_names = list(l3_class_indexes.keys())

l3gt_names = []
l3pred_image_names = []
l3pred_names = []
for yi, ymi in enumerate(yoloobject_img_names):
    l3gt = l3gt_pdf[1][yi]
    l3pred_image = imgnames[imgnames_clipped.index(ymi)]
    l3pred = l3_class_indexes_names[pdffk[1][imgnames_clipped.index(ymi)]]
    
    #os.system('cp /scratch/prafful/test_dumps_orig/'+l3pred_image+' /scratch/prafful/test_dumps_orig_gtvalid/')
    l3pred_image_names.append(l3pred_image)
    l3gt_names.append(l3gt)
    l3pred_names.append(l3pred)

pd.DataFrame({0:l3pred_image_names, 1:l3gt_names, 2:l3pred_names}).to_csv("test_dumps_orig_gtvalid_wpred_list2.txt",sep='\t',header=False,index=False)

car_Toyota_Innova


In [19]:
pname = ''
l2_names_order = []
l3_l2_mapper = []
for l3i in l3_class_indexes_names:
    if(pname!='_'.join(l3i.split('_')[:2])):
        pname = '_'.join(l3i.split('_')[:2])
        l2_names_order.append(pname)
    l3_l2_mapper.append(len(l2_names_order)-1)
l3_l2_mapper

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 7,
 8,
 8,
 8,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 26,
 26,
 26,
 27,
 27,
 27,
 27,
 28,
 28,
 29,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 32,
 32,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 34,
 35,
 36,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 38,
 38,
 38,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 41,
 42,
 42,
 42,
 43,
 43,
 43,
 43,
 44,
 45,
 46,
 46,
 46,
 47,
 47,
 47

In [7]:
pdfik = pd.read_csv("test_dumps_orig_gtvalid_wpred_list2.txt",sep='\t',header=None)
tmatches = 0
for pfi in range(pdfik.shape[0]):
    if(pdfik[1][pfi]==pdfik[2][pfi]):
        tmatches+=1
print(tmatches)
pdfik.head()

3496


,0,1,2
0,mainbatchi0_imgbatchi0_predi1_1.png,car_Toyota_Innova,car_Toyota_Innova
1,mainbatchi0_imgbatchi0_predi2_2.png,car_TataMotors_Zest,car_TataMotors_Zest
2,mainbatchi0_imgbatchi0_predi0_0.png,autorickshaw_Bajaj_Bajaj,autorickshaw_Bajaj_Bajaj
3,mainbatchi0_imgbatchi1_predi0_25.png,car_TataMotors_Zest,car_TataMotors_Zest
4,mainbatchi0_imgbatchi2_predi2_36.png,car_Honda_Amaze,car_Honda_Amaze


In [27]:
import pandas as pd
modify_csv = "test_dumps_orig_list2_outl1.txt"
tempdf = pd.read_csv(modify_csv,sep='\t',header=None)
tempdf.head()
tnames = [ti.split('/')[-1] for ti in tempdf[0]]
pd.DataFrame({0:tnames, 1:list(tempdf[1]), 2:list(tempdf[2])}).to_csv(modify_csv,sep='\t',header=False,index=False)
tempdf = pd.read_csv(modify_csv,sep='\t',header=None)
tempdf.head()


,0,1,2
0,mainbatchi0_imgbatchi0_predi0_0.png,0,2
1,mainbatchi0_imgbatchi0_predi1_1.png,2,2
2,mainbatchi0_imgbatchi0_predi2_2.png,2,2
3,mainbatchi0_imgbatchi0_predi3_3.png,4,2
4,mainbatchi0_imgbatchi0_predi4_4.png,0,2


In [1]:
import numpy as np, cv2
import pandas as pd
import pickle

with open('/home2/praffulkumar/cp_wks/yolov5/v5l_fgd_l1_test_stats.pkl', 'rb') as file:
    # A new file will be created
    stats = pickle.load(file)
with open('/home2/praffulkumar/cp_wks/yolov5/v5l_fgd_l3_test_stats.pkl', 'rb') as file3:
    # A new file will be created
    stats3 = pickle.load(file3)
# stats = (correct, conf, pcls, tcls)
print(stats[0].shape, stats[1].shape, stats[2].shape, stats[3].shape )

imgnames = pd.read_csv("test_dumps_orig_list2.txt",sep='\t',header=None)[0]
img_idxs = [int(imi.split('_')[-1][:-4]) for imi in imgnames]

l3gt_pdf = pd.read_csv("l3gt_foryoloobjects.csv")
yoloobject_img_names = list(l3gt_pdf['yoloobject_img_name'])

l1_names = [ 'motorcycle', 'car', 'scooter', 'autorickshaw', 'truck', 'mini-bus', 'bus' ]
l3_class_indexes = {'autorickshaw_Atul_Atul': 1, 'autorickshaw_autorickshaw_autorickshaw': 2, 'autorickshaw_Bajaj_Bajaj': 3, 'autorickshaw_Mahindra_Mahindra': 4, 'autorickshaw_Piaggio_Piaggio': 5, 'autorickshaw_TVS_TVS': 6, 'bus_bus_bus': 7, 'car_Audi_A3': 8, 'car_Audi_Q3': 9, 'car_Bmw_2-Series-220d': 10, 'car_Bmw_3-Series': 11, 'car_Bmw_X1': 12, 'car_car_car': 13, 'car_Chevrolet_Aveo': 14, 'car_Chevrolet_Beat': 15, 'car_Chevrolet_Cruze': 16, 'car_Chevrolet_Enjoy': 17, 'car_Chevrolet_Spark': 18, 'car_Chevrolet_Tavera': 19, 'car_Fiat_Linea': 20, 'car_Fiat_PuntoEvo': 21, 'car_Force_TraxToofan': 22, 'car_Ford_Aspire': 23, 'car_Ford_Ecosport': 24, 'car_Ford_EcoSportTitanium': 25, 'car_Ford_Everest': 26, 'car_Ford_Fiesta': 27, 'car_Ford_Figo': 28, 'car_Ford_Ikon': 29, 'car_Hindustan_Ambassador': 30, 'car_Honda_Accord': 31, 'car_Honda_Amaze': 32, 'car_Honda_Brio': 33, 'car_Honda_Brv': 34, 'car_Honda_City': 35, 'car_Honda_Civic': 36, 'car_Honda_Cr-V': 37, 'car_Honda_Jazz': 38, 'car_Honda_Wr-V': 39, 'car_Hyundai_Accent': 40, 'car_Hyundai_Aura': 41, 'car_Hyundai_Creta': 42, 'car_Hyundai_Eon': 43, 'car_Hyundai_I10': 44, 'car_Hyundai_I20': 45, 'car_Hyundai_Santro': 46, 'car_Hyundai_Verna': 47, 'car_Hyundai_Xcent': 48, 'car_Jeep_Compass': 49, 'car_Jeep_Wrangler': 50, 'car_Mahindra_Bolero': 51, 'car_Mahindra_Reva': 52, 'car_Mahindra_Scorpio': 53, 'car_Mahindra_TUV300': 54, 'car_Mahindra_Verito': 55, 'car_Mahindra_XUV500': 56, 'car_Mahindra_Xylo': 57, 'car_MarutiSuzuki_1000': 58, 'car_MarutiSuzuki_Alto800': 59, 'car_MarutiSuzuki_AltoK10': 60, 'car_MarutiSuzuki_Baleno': 61, 'car_MarutiSuzuki_Celerio': 62, 'car_MarutiSuzuki_Ciaz': 63, 'car_MarutiSuzuki_Dzire': 64, 'car_MarutiSuzuki_Eeco': 65, 'car_MarutiSuzuki_Ertiga': 66, 'car_MarutiSuzuki_Esteem2000': 67, 'car_MarutiSuzuki_Ignis': 68, 'car_MarutiSuzuki_Omni': 69, 'car_MarutiSuzuki_Ritz': 70, 'car_MarutiSuzuki_S-Cross': 71, 'car_MarutiSuzuki_Swift': 72, 'car_MarutiSuzuki_SX4': 73, 'car_MarutiSuzuki_VitaraBrezza': 74, 'car_MarutiSuzuki_WagonR': 75, 'car_MarutiSuzuki_Zen': 76, 'car_Mercedes-Benz_A-Class': 77, 'car_Mercedes-Benz_AmgGt4-DoorCoupe': 78, 'car_Mercedes-Benz_C-Class': 79, 'car_Mercedes-Benz_E-Class': 80, 'car_Mercedes-Benz_G-Class': 81, 'car_Mercedes-Benz_Gla-Class': 82, 'car_Mercedes-Benz_Gls': 83, 'car_Mercedes-Benz_S-Class': 84, 'car_Mitsubishi_Lancer': 85, 'car_Nissan_Micra': 86, 'car_Nissan_Sunny': 87, 'car_Nissan_Terrano': 88, 'car_Renault_Duster': 89, 'car_Renault_Kwid': 90, 'car_Renault_Lodgy': 91, 'car_Renault_Logan': 92, 'car_Renault_Scala': 93, 'car_Skoda_Fabia': 94, 'car_Skoda_Octavia': 95, 'car_Skoda_Rapid': 96, 'car_Skoda_Superb': 97, 'car_TataMotors_Hexa': 98, 'car_TataMotors_Indica': 99, 'car_TataMotors_Indigo': 100,'car_TataMotors_Nano': 101, 'car_TataMotors_Nexon': 102, 'car_TataMotors_Safari': 103, 'car_TataMotors_Sumo': 104, 'car_TataMotors_Tiago': 105, 'car_TataMotors_Tigor': 106, 'car_TataMotors_Zest': 107, 'car_Toyota_Corolla': 108, 'car_Toyota_Etios': 109, 'car_Toyota_EtiosLiva': 110, 'car_Toyota_Fortuner': 111, 'car_Toyota_Innova': 112, 'car_Toyota_Qualis': 113, 'car_Volkswagen_Ameo': 114, 'car_Volkswagen_Jetta': 115, 'car_Volkswagen_Polo': 116, 'car_Volkswagen_Vento': 117, 'car_Volvo_Xc40': 118, 'car_Volvo_Xc60': 119, 'mini-bus_mini-bus_mini-bus': 120, 'motorcycle_Bajaj_Avenger': 121, 'motorcycle_Bajaj_CT100': 122, 'motorcycle_Bajaj_Discover': 123, 'motorcycle_Bajaj_Discover100': 124, 'motorcycle_Bajaj_Discover110': 125, 'motorcycle_Bajaj_Discover125': 126, 'motorcycle_Bajaj_Discover135': 127, 'motorcycle_Bajaj_Platina': 128, 'motorcycle_Bajaj_Pulsar150': 129, 'motorcycle_Bajaj_Pulsar180': 130, 'motorcycle_Bajaj_Pulsar200': 131, 'motorcycle_Bajaj_Pulsar220F': 132, 'motorcycle_Bajaj_PulsarNS200': 133, 'motorcycle_Bajaj_PulsarRS200': 134, 'motorcycle_Bajaj_V12': 135, 'motorcycle_Bajaj_V15': 136, 'motorcycle_Hero_Glamour': 137, 'motorcycle_Hero_HFDeluxe': 138, 'motorcycle_Hero_Hunk': 139, 'motorcycle_Hero_Passion': 140, 'motorcycle_Hero_PassionPlus': 141, 'motorcycle_Hero_PassionPro': 142, 'motorcycle_Hero_Splendor': 143, 'motorcycle_Hero_SuperSplendor': 144, 'motorcycle_Hero_XPulse200': 145, 'motorcycle_HeroHonda_CBZ': 146, 'motorcycle_HeroHonda_SplendorNXG': 147, 'motorcycle_Honda_CBHornet160R': 148, 'motorcycle_Honda_CBTwister': 149, 'motorcycle_Honda_Karizma': 150, 'motorcycle_Honda_KarizmaZMR': 151, 'motorcycle_Honda_Shine': 152, 'motorcycle_Honda_SP125': 153, 'motorcycle_Honda_StunnerCBF': 154, 'motorcycle_Honda_Unicorn': 155, 'motorcycle_KTM_Duke': 156, 'motorcycle_Mahindra_Centuro': 157, 'motorcycle_motorcycle_motorcycle': 158, 'motorcycle_RoyalEnfield_Bullet350': 159, 'motorcycle_RoyalEnfield_Bullet500': 160, 'motorcycle_RoyalEnfield_Classic350': 161, 'motorcycle_RoyalEnfield_Classic500': 162, 'motorcycle_RoyalEnfield_ContinentalGT650': 163, 'motorcycle_RoyalEnfield_Interceptor650': 164, 'motorcycle_RoyalEnfield_Meteor350': 165, 'motorcycle_RoyalEnfield_Thunderbird350': 166, 'motorcycle_RoyalEnfield_Thunderbird350X': 167, 'motorcycle_Suzuki_Gixxer': 168, 'motorcycle_Suzuki_Samurai': 169, 'motorcycle_Suzuki_Slingshot': 170, 'motorcycle_TVS_ApacheRR310': 171, 'motorcycle_TVS_ApacheRTR160': 172, 'motorcycle_TVS_ApacheRTR200': 173, 'motorcycle_TVS_Excel100': 174, 'motorcycle_TVS_ExcelHeavyDuty': 175, 'motorcycle_TVS_Sport': 176, 'motorcycle_TVS_StarCityPlus': 177, 'motorcycle_TVS_Victor': 178, 'motorcycle_TVS_XL100': 179, 'motorcycle_Yamaha_Crux': 180, 'motorcycle_Yamaha_Fazer': 181, 'motorcycle_Yamaha_FZ25': 182, 'motorcycle_Yamaha_FZS-FI': 183, 'motorcycle_Yamaha_FZ-V3': 184, 'motorcycle_Yamaha_Libero': 185, 'motorcycle_Yamaha_R15': 186, 'motorcycle_Yamaha_RX100': 187, 'scooter_Bajaj_Chetak': 188, 'scooter_Hero_Duet': 189, 'scooter_Hero_Maestro': 190, 'scooter_Hero_Pleasure': 191, 'scooter_Honda_Activa': 192, 'scooter_Honda_Aviator': 193, 'scooter_Honda_Dio': 194, 'scooter_Honda_Grazia': 195, 'scooter_Mahindra_Gusto': 196, 'scooter_scooter_scooter': 197, 'scooter_Suzuki_Access': 198, 'scooter_Suzuki_Burgman': 199, 'scooter_Suzuki_Swish': 200, 'scooter_TVS_Jupiter': 201, 'scooter_TVS_Ntorq': 202, 'scooter_TVS_Pep': 203, 'scooter_TVS_Streak': 204,'scooter_TVS_Wego': 205, 'scooter_TVS_Zest': 206, 'scooter_Vespa_VXL125': 207, 'scooter_Vespa_ZX125': 208, 'scooter_Yamaha_Fascino125': 209, 'scooter_Yamaha_RayZR': 210, 'truck_AshokLeyland_AshokLeyland': 211, 'truck_BharatBenz_BharatBenz': 212, 'truck_Eicher_Eicher': 213, 'truck_Mahindra_Mahindra': 214, 'truck_SML_SML': 215, 'truck_Tata_Tata': 216, 'truck_truck_truck': 217}
l3_class_indexes_names = list(l3_class_indexes.keys())

l1Tl3F = np.multiply(stats[0][:,0], stats3[0][:,0]==False)
l1Tl3F_idxs = np.where(l1Tl3F)[0]

l3_gts_arr = []
l3_pred_imnm = []
l3_gt_imnm = []
for li in l1Tl3F_idxs:
    orig_img_nm = imgnames[img_idxs.index(li)]
    obj_key_name = '_'.join(orig_img_nm.split('_')[:3])
    l3gt_idx = l3gt_pdf['l3gt_foryoloobject'][yoloobject_img_names.index(obj_key_name)]
    
    gt_obj_name = "/scratch/prafful/orig_gt_object_images/"+obj_key_name+".png"
    yolo_obj_addr = "/scratch/prafful/test_dumps_orig/"+orig_img_nm
    canvas_gt_yolo_obj_addr = 'l1Tl3F_imgs/u'+str(li)+'-l1label_'+l1_names[int(stats[2][li])]+'-l3pred_'+l3_class_indexes_names[int(stats3[2][li])]+'-l3gt_'+l3_class_indexes_names[l3gt_idx]+'.png'
    
    l3_gts_arr.append(l3_class_indexes_names[l3gt_idx])
    l3_pred_imnm.append(orig_img_nm)
    l3_gt_imnm.append(obj_key_name+".png")
    
# pd.DataFrame({0:l3_pred_imnm, 1:l3_gts_arr}).to_csv("/scratch/prafful/test_dumps_orig_fails_list.txt",sep='\t',header=False,index=False)
# pd.DataFrame({0:l3_gt_imnm, 1:l3_gts_arr}).to_csv("/scratch/prafful/orig_gt_object_images_fails_list.txt",sep='\t',header=False,index=False)
    
    im2 = cv2.imread(gt_obj_name)
    im1 = cv2.imread(yolo_obj_addr)
    
#     cv2.imwrite(yolo_obj_addr.replace('test_dumps_orig','test_dumps_orig_fails'),im1)
#     cv2.imwrite(gt_obj_name.replace('orig_gt_object_images','orig_gt_object_images_fails'),im2)
    
    hmax, wmax = max(im1.shape[0],im2.shape[0]), max(im1.shape[1],im2.shape[1])
    canvas_gt_yolo_img = np.ones((hmax,wmax*2,3),dtype='uint8')*255
    canvas_gt_yolo_img[:im1.shape[0],:im1.shape[1]] = im1
    canvas_gt_yolo_img[:im2.shape[0],wmax:wmax+im2.shape[1]] = im2
    cv2.imwrite(canvas_gt_yolo_obj_addr,canvas_gt_yolo_img)
    
    print(gt_obj_name, yolo_obj_addr, canvas_gt_yolo_obj_addr)
    
#     break
## what is the target mAP and what is the diff in classification accuracies
## what is the L2 and L1 mAP

(39650, 10) (39650,) (39650,) (4891,)
/scratch/prafful/orig_gt_object_images/mainbatchi0_imgbatchi6_predi2.png /scratch/prafful/test_dumps_orig/mainbatchi0_imgbatchi6_predi2_167.png l1Tl3F_imgs/u167-l1label_car-l3pred_car_MarutiSuzuki_Baleno-l3gt_car_car_car.png
/scratch/prafful/orig_gt_object_images/mainbatchi0_imgbatchi6_predi3.png /scratch/prafful/test_dumps_orig/mainbatchi0_imgbatchi6_predi3_168.png l1Tl3F_imgs/u168-l1label_scooter-l3pred_scooter_Suzuki_Access-l3gt_scooter_TVS_Pep.png
/scratch/prafful/orig_gt_object_images/mainbatchi0_imgbatchi6_predi21.png /scratch/prafful/test_dumps_orig/mainbatchi0_imgbatchi6_predi21_186.png l1Tl3F_imgs/u186-l1label_car-l3pred_car_MarutiSuzuki_Ertiga-l3gt_car_Toyota_Innova.png
/scratch/prafful/orig_gt_object_images/mainbatchi0_imgbatchi8_predi1.png /scratch/prafful/test_dumps_orig/mainbatchi0_imgbatchi8_predi1_238.png l1Tl3F_imgs/u238-l1label_truck-l3pred_truck_Tata_Tata-l3gt_truck_truck_truck.png
/scratch/prafful/orig_gt_object_images/mainbatch

In [13]:
# save a canvas of original image and its corresponding gt label alongwith the yolopred image and the pred labels

'mainbatchi0_imgbatchi0_predi2_2.png'

In [7]:
import numpy as np
aka = stats[2][stats[0][:,0]]
np.min(aka),np.max(aka),np.unique(aka).shape,aka.shape,np.unique(stats[3]).shape

(1.0, 216.0, (114,), (4642,), (170,))

In [8]:
l3_class_indexes = {'autorickshaw_Atul_Atul': 1, 'autorickshaw_autorickshaw_autorickshaw': 2, 'autorickshaw_Bajaj_Bajaj': 3, 'autorickshaw_Mahindra_Mahindra': 4, 'autorickshaw_Piaggio_Piaggio': 5, 'autorickshaw_TVS_TVS': 6, 'bus_bus_bus': 7, 'car_Audi_A3': 8, 'car_Audi_Q3': 9, 'car_Bmw_2-Series-220d': 10, 'car_Bmw_3-Series': 11, 'car_Bmw_X1': 12, 'car_car_car': 13, 'car_Chevrolet_Aveo': 14, 'car_Chevrolet_Beat': 15, 'car_Chevrolet_Cruze': 16, 'car_Chevrolet_Enjoy': 17, 'car_Chevrolet_Spark': 18, 'car_Chevrolet_Tavera': 19, 'car_Fiat_Linea': 20, 'car_Fiat_PuntoEvo': 21, 'car_Force_TraxToofan': 22, 'car_Ford_Aspire': 23, 'car_Ford_Ecosport': 24, 'car_Ford_EcoSportTitanium': 25, 'car_Ford_Everest': 26, 'car_Ford_Fiesta': 27, 'car_Ford_Figo': 28, 'car_Ford_Ikon': 29, 'car_Hindustan_Ambassador': 30, 'car_Honda_Accord': 31, 'car_Honda_Amaze': 32, 'car_Honda_Brio': 33, 'car_Honda_Brv': 34, 'car_Honda_City': 35, 'car_Honda_Civic': 36, 'car_Honda_Cr-V': 37, 'car_Honda_Jazz': 38, 'car_Honda_Wr-V': 39, 'car_Hyundai_Accent': 40, 'car_Hyundai_Aura': 41, 'car_Hyundai_Creta': 42, 'car_Hyundai_Eon': 43, 'car_Hyundai_I10': 44, 'car_Hyundai_I20': 45, 'car_Hyundai_Santro': 46, 'car_Hyundai_Verna': 47, 'car_Hyundai_Xcent': 48, 'car_Jeep_Compass': 49, 'car_Jeep_Wrangler': 50, 'car_Mahindra_Bolero': 51, 'car_Mahindra_Reva': 52, 'car_Mahindra_Scorpio': 53, 'car_Mahindra_TUV300': 54, 'car_Mahindra_Verito': 55, 'car_Mahindra_XUV500': 56, 'car_Mahindra_Xylo': 57, 'car_MarutiSuzuki_1000': 58, 'car_MarutiSuzuki_Alto800': 59, 'car_MarutiSuzuki_AltoK10': 60, 'car_MarutiSuzuki_Baleno': 61, 'car_MarutiSuzuki_Celerio': 62, 'car_MarutiSuzuki_Ciaz': 63, 'car_MarutiSuzuki_Dzire': 64, 'car_MarutiSuzuki_Eeco': 65, 'car_MarutiSuzuki_Ertiga': 66, 'car_MarutiSuzuki_Esteem2000': 67, 'car_MarutiSuzuki_Ignis': 68, 'car_MarutiSuzuki_Omni': 69, 'car_MarutiSuzuki_Ritz': 70, 'car_MarutiSuzuki_S-Cross': 71, 'car_MarutiSuzuki_Swift': 72, 'car_MarutiSuzuki_SX4': 73, 'car_MarutiSuzuki_VitaraBrezza': 74, 'car_MarutiSuzuki_WagonR': 75, 'car_MarutiSuzuki_Zen': 76, 'car_Mercedes-Benz_A-Class': 77, 'car_Mercedes-Benz_AmgGt4-DoorCoupe': 78, 'car_Mercedes-Benz_C-Class': 79, 'car_Mercedes-Benz_E-Class': 80, 'car_Mercedes-Benz_G-Class': 81, 'car_Mercedes-Benz_Gla-Class': 82, 'car_Mercedes-Benz_Gls': 83, 'car_Mercedes-Benz_S-Class': 84, 'car_Mitsubishi_Lancer': 85, 'car_Nissan_Micra': 86, 'car_Nissan_Sunny': 87, 'car_Nissan_Terrano': 88, 'car_Renault_Duster': 89, 'car_Renault_Kwid': 90, 'car_Renault_Lodgy': 91, 'car_Renault_Logan': 92, 'car_Renault_Scala': 93, 'car_Skoda_Fabia': 94, 'car_Skoda_Octavia': 95, 'car_Skoda_Rapid': 96, 'car_Skoda_Superb': 97, 'car_TataMotors_Hexa': 98, 'car_TataMotors_Indica': 99, 'car_TataMotors_Indigo': 100,'car_TataMotors_Nano': 101, 'car_TataMotors_Nexon': 102, 'car_TataMotors_Safari': 103, 'car_TataMotors_Sumo': 104, 'car_TataMotors_Tiago': 105, 'car_TataMotors_Tigor': 106, 'car_TataMotors_Zest': 107, 'car_Toyota_Corolla': 108, 'car_Toyota_Etios': 109, 'car_Toyota_EtiosLiva': 110, 'car_Toyota_Fortuner': 111, 'car_Toyota_Innova': 112, 'car_Toyota_Qualis': 113, 'car_Volkswagen_Ameo': 114, 'car_Volkswagen_Jetta': 115, 'car_Volkswagen_Polo': 116, 'car_Volkswagen_Vento': 117, 'car_Volvo_Xc40': 118, 'car_Volvo_Xc60': 119, 'mini-bus_mini-bus_mini-bus': 120, 'motorcycle_Bajaj_Avenger': 121, 'motorcycle_Bajaj_CT100': 122, 'motorcycle_Bajaj_Discover': 123, 'motorcycle_Bajaj_Discover100': 124, 'motorcycle_Bajaj_Discover110': 125, 'motorcycle_Bajaj_Discover125': 126, 'motorcycle_Bajaj_Discover135': 127, 'motorcycle_Bajaj_Platina': 128, 'motorcycle_Bajaj_Pulsar150': 129, 'motorcycle_Bajaj_Pulsar180': 130, 'motorcycle_Bajaj_Pulsar200': 131, 'motorcycle_Bajaj_Pulsar220F': 132, 'motorcycle_Bajaj_PulsarNS200': 133, 'motorcycle_Bajaj_PulsarRS200': 134, 'motorcycle_Bajaj_V12': 135, 'motorcycle_Bajaj_V15': 136, 'motorcycle_Hero_Glamour': 137, 'motorcycle_Hero_HFDeluxe': 138, 'motorcycle_Hero_Hunk': 139, 'motorcycle_Hero_Passion': 140, 'motorcycle_Hero_PassionPlus': 141, 'motorcycle_Hero_PassionPro': 142, 'motorcycle_Hero_Splendor': 143, 'motorcycle_Hero_SuperSplendor': 144, 'motorcycle_Hero_XPulse200': 145, 'motorcycle_HeroHonda_CBZ': 146, 'motorcycle_HeroHonda_SplendorNXG': 147, 'motorcycle_Honda_CBHornet160R': 148, 'motorcycle_Honda_CBTwister': 149, 'motorcycle_Honda_Karizma': 150, 'motorcycle_Honda_KarizmaZMR': 151, 'motorcycle_Honda_Shine': 152, 'motorcycle_Honda_SP125': 153, 'motorcycle_Honda_StunnerCBF': 154, 'motorcycle_Honda_Unicorn': 155, 'motorcycle_KTM_Duke': 156, 'motorcycle_Mahindra_Centuro': 157, 'motorcycle_motorcycle_motorcycle': 158, 'motorcycle_RoyalEnfield_Bullet350': 159, 'motorcycle_RoyalEnfield_Bullet500': 160, 'motorcycle_RoyalEnfield_Classic350': 161, 'motorcycle_RoyalEnfield_Classic500': 162, 'motorcycle_RoyalEnfield_ContinentalGT650': 163, 'motorcycle_RoyalEnfield_Interceptor650': 164, 'motorcycle_RoyalEnfield_Meteor350': 165, 'motorcycle_RoyalEnfield_Thunderbird350': 166, 'motorcycle_RoyalEnfield_Thunderbird350X': 167, 'motorcycle_Suzuki_Gixxer': 168, 'motorcycle_Suzuki_Samurai': 169, 'motorcycle_Suzuki_Slingshot': 170, 'motorcycle_TVS_ApacheRR310': 171, 'motorcycle_TVS_ApacheRTR160': 172, 'motorcycle_TVS_ApacheRTR200': 173, 'motorcycle_TVS_Excel100': 174, 'motorcycle_TVS_ExcelHeavyDuty': 175, 'motorcycle_TVS_Sport': 176, 'motorcycle_TVS_StarCityPlus': 177, 'motorcycle_TVS_Victor': 178, 'motorcycle_TVS_XL100': 179, 'motorcycle_Yamaha_Crux': 180, 'motorcycle_Yamaha_Fazer': 181, 'motorcycle_Yamaha_FZ25': 182, 'motorcycle_Yamaha_FZS-FI': 183, 'motorcycle_Yamaha_FZ-V3': 184, 'motorcycle_Yamaha_Libero': 185, 'motorcycle_Yamaha_R15': 186, 'motorcycle_Yamaha_RX100': 187, 'scooter_Bajaj_Chetak': 188, 'scooter_Hero_Duet': 189, 'scooter_Hero_Maestro': 190, 'scooter_Hero_Pleasure': 191, 'scooter_Honda_Activa': 192, 'scooter_Honda_Aviator': 193, 'scooter_Honda_Dio': 194, 'scooter_Honda_Grazia': 195, 'scooter_Mahindra_Gusto': 196, 'scooter_scooter_scooter': 197, 'scooter_Suzuki_Access': 198, 'scooter_Suzuki_Burgman': 199, 'scooter_Suzuki_Swish': 200, 'scooter_TVS_Jupiter': 201, 'scooter_TVS_Ntorq': 202, 'scooter_TVS_Pep': 203, 'scooter_TVS_Streak': 204,'scooter_TVS_Wego': 205, 'scooter_TVS_Zest': 206, 'scooter_Vespa_VXL125': 207, 'scooter_Vespa_ZX125': 208, 'scooter_Yamaha_Fascino125': 209, 'scooter_Yamaha_RayZR': 210, 'truck_AshokLeyland_AshokLeyland': 211, 'truck_BharatBenz_BharatBenz': 212, 'truck_Eicher_Eicher': 213, 'truck_Mahindra_Mahindra': 214, 'truck_SML_SML': 215, 'truck_Tata_Tata': 216, 'truck_truck_truck': 217}
l3_class_indexes_names = list(l3_class_indexes.keys())

from pathlib import Path
from utils.metrics import ConfusionMatrix, ap_per_class, box_iou
## eval using stats variable first then proceed to next cell
###### Run this for final output
save_dir=Path('')
plots=False
l3_names_dict = {}
for li, ci in enumerate(l3_class_indexes_names):
    l3_names_dict[li] = ci

tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=l3_names_dict)
p.mean(), r.mean(), ap[:, 0].mean()

# f-score wala best i ka value: 461 out of  1000
# (0.014658523305032133, 0.8166648909026482, 0.3274881928637568)
# 0.6146318498801795, 0.6350555170645438, 0.5158146712051431

(0.5917218315707693, 0.5460939556408839, 0.5518570085539264)

In [6]:
ap.shape

(170, 10)

In [18]:
np.sum(ap[:,0]>0)

108